# Derleme

In [1]:
#Altın fiyatını çekme
import requests

def make_gapi_request():
    api_key = "goldapi-13te5osm652zpew-io"  # Buraya doğru API anahtarını gir
    symbol = "XAU"
    curr = "USD"
    
    url = f"https://www.goldapi.io/api/{symbol}/{curr}"
    
    headers = {
        "x-access-token": api_key,
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Eğer hata alırsak burada Exception fırlatır
        
        # API yanıtını JSON olarak işle
        data = response.json()
        gold_price = data.get("price")  # Altın fiyatını al
        return gold_price
    except requests.exceptions.RequestException as e:
        print("API Hatası:", str(e))
        return None

# Fonksiyonu çağır ve sonucu yazdır
gold_price = make_gapi_request()
if gold_price:
    print(f"Anlık Altın Fiyatı (Gram): {gold_price} USD")
else:
    print("Altın fiyatı alınamadı.")


Anlık Altın Fiyatı (Gram): 2761.065 USD


In [2]:
#Borsa hisselerini çekme
import yfinance as yf

def get_stock_prices():
    tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "BRK-B", "NVDA", "JPM", "V"]  # İlk 10 hisse
    stock_data = {}
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        stock_data[ticker] = stock.history(period="1d")['Close'].iloc[-1]  # Son kapanış fiyatı
    return stock_data

stocks = get_stock_prices()
for ticker, price in stocks.items():
    print(f"{ticker}: {price} USD")


AAPL: 222.63999938964844 USD
MSFT: 428.5 USD
GOOGL: 198.0500030517578 USD
AMZN: 230.7100067138672 USD
META: 616.4600219726562 USD
TSLA: 424.07000732421875 USD
BRK-B: 468.57000732421875 USD
NVDA: 140.8300018310547 USD
JPM: 263.0299987792969 USD
V: 323.6300048828125 USD


In [3]:
#########Birleştirilmiş hal:

# Altın fiyatını çekme
import requests
import yfinance as yf

def get_gold_price():
    api_key = "goldapi-13te5osm652zpew-io"  # Buraya doğru API anahtarını gir
    symbol = "XAU"
    curr = "USD"
    
    url = f"https://www.goldapi.io/api/{symbol}/{curr}"
    
    headers = {
        "x-access-token": api_key,
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Eğer hata alırsak burada Exception fırlatır
        
        # API yanıtını JSON olarak işle
        data = response.json()
        gold_price = data.get("price")  # Altın fiyatını al
        return gold_price
    except requests.exceptions.RequestException as e:
        print("API Hatası:", str(e))
        return None

# Borsa hisselerini çekme
def get_stock_prices():
    tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "BRK-B", "NVDA", "JPM", "V"]  # İlk 10 hisse
    stock_data = {}
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        stock_data[ticker] = stock.history(period="1d")['Close'].iloc[-1]  # Son kapanış fiyatı
    return stock_data

# Yatırım önerisi hesaplama
def calculate_investment(amount, gold_price, stock_prices, currency_rate):
    # Altın yatırımı
    gold_units = amount / (gold_price * currency_rate)  # Altın fiyatını TL'ye çevirip hesapla
    gold_return = gold_units * (gold_price * currency_rate * 1.01)  # %1 değer artışı varsayımı
    
    # Hisse yatırımı
    stock_returns = {}
    for ticker, price in stock_prices.items():
        stock_units = amount / (price * currency_rate)  # TL'yi USD'ye çevirerek birim sayısını hesapla
        stock_returns[ticker] = stock_units * (price * 1.02)  # %2 değer artışı varsayımı
    
    return gold_return, stock_returns




In [4]:
 #güncel kuru çekme

def get_currency_rate():
    api_key = "01f5e53584c79bfac0ecedca37f6b9ce"  # Buraya API anahtarını gir
    url = "https://open.er-api.com/v6/latest/USD"  # API'nin endpoint'i

    headers = {
        "Authorization": f"Bearer {api_key}",  # API anahtarını Authorization başlığına ekle
        "Content-Type": "application/json"
    }

    try:
        response = requests.get(url, headers=headers)  # headers ile istek yap
        response.raise_for_status()  # Hata varsa yakalar
        
        data = response.json()  # JSON formatında yanıt al
        return data["rates"]["TRY"]  # USD/TRY kuru
    except requests.exceptions.RequestException as e:
        print("Döviz kuru alınamadı:", str(e))
        return None

# Test etmek için:
currency_rate = get_currency_rate()
if currency_rate:
    print(f"Güncel USD/TRY kuru: {currency_rate}")
else:
    print("Kur bilgisi alınamadı.")



Güncel USD/TRY kuru: 35.628548


In [5]:
#Ana fonksiyon
def main():
    #Kullanıcıdan yatırım miktarının alınması 
    amount = float(input("Yatırım yapılacak miktar (TL): "))
    
    #Güncel kur al 
    currency_rate = get_currency_rate()
    if not currency_rate:
        print("Döviz kuru alınamadı, program sonlandırıldı.")
    
    #Altın fiyatını al 
    gold_price = get_gold_price()
    if not gold_price:
        print("Altın fiyatı alınamadı, program sonlandırıldı.")
        return
    
    #Hisse fiyatını al 
    stock_prices = get_stock_prices()
    
    #Öneriyi hesapla
    gold_return, stock_returns = calculate_investment(amount, gold_price, stock_prices, currency_rate)
    
    # Karşılaştırma ve öneri
    best_stock = max(stock_returns, key=stock_returns.get)
    if gold_return > stock_returns[best_stock]:
        print(f"Altına yatırım yapmak daha mantıklı. Tahmini getiri: {gold_return:.2f} TL")
    else:
        print(f"{best_stock} hissesine yatırım yapmak daha mantıklı. Tahmini getiri: {stock_returns[best_stock]:.2f} TL")

# Programı başlat
if __name__ == "__main__":
    main()

Altına yatırım yapmak daha mantıklı. Tahmini getiri: 101.00 TL


# Yalnızca SP&10 için 

In [6]:
def calculate_stock_returns(amount, stock_prices, potential_increase=0.05, potential_decrease=0.03):
    """
    Her bir hisse için tahmini getiri ve götürüyü hesaplar.
    Args:
        amount (float): Yatırım miktarı (TL cinsinden).
        stock_prices (dict): Hisse fiyatları (USD cinsinden).
        potential_increase (float): Tahmini fiyat artış oranı (örneğin, %5 = 0.05).
        potential_decrease (float): Tahmini fiyat düşüş oranı (örneğin, %3 = 0.03).
    Returns:
        dict: Her hisse için tahmini getiri ve götürü.
    """
    currency_rate = get_currency_rate()  # USD/TRY kuru
    if not currency_rate:
        print("Döviz kuru alınamadı, program sonlandırılıyor.")
        return None

    stock_results = {}
    for ticker, price in stock_prices.items():
        # Hisse başına alınabilecek birim sayısı
        units = amount / (price * currency_rate)
        # Tahmini getiri ve götürü hesaplama
        expected_increase = units * (price * (1 + potential_increase) * currency_rate)
        expected_decrease = units * (price * (1 - potential_decrease) * currency_rate)
        
        stock_results[ticker] = {
            "Tahmini Getiri (TL)": expected_increase,
            "Tahmini Götürü (TL)": expected_decrease,
            "Hisse Birim Fiyatı (USD)": price,
            "Hisse Birim Fiyatı (TL)": price * currency_rate
        }
    
    return stock_results

In [7]:
def main():
    """
    Ana fonksiyon: Kullanıcıdan yatırım miktarını alır ve tahmini getiri/götürüyü hesaplar.
    """
    amount = float(input("Yatırım yapmak istediğiniz miktar (TL): "))
    
    # Hisse fiyatlarını al
    stock_prices = get_stock_prices()
    
    # Her hisse için tahmini getiri ve götürüyü hesapla
    stock_results = calculate_stock_returns(amount, stock_prices)
    
    # Sonuçları ekrana yazdır
    if stock_results:
        print("\nTahmini Getiri ve Götürü:")
        for ticker, data in stock_results.items():
            print(f"\nHisse: {ticker}")
            for key, value in data.items():
                print(f"{key}: {value:.2f} TL")

# Programı çalıştır
if __name__ == "__main__":
    main()


Tahmini Getiri ve Götürü:

Hisse: AAPL
Tahmini Getiri (TL): 105.00 TL
Tahmini Götürü (TL): 97.00 TL
Hisse Birim Fiyatı (USD): 222.64 TL
Hisse Birim Fiyatı (TL): 7932.34 TL

Hisse: MSFT
Tahmini Getiri (TL): 105.00 TL
Tahmini Götürü (TL): 97.00 TL
Hisse Birim Fiyatı (USD): 428.50 TL
Hisse Birim Fiyatı (TL): 15266.83 TL

Hisse: GOOGL
Tahmini Getiri (TL): 105.00 TL
Tahmini Götürü (TL): 97.00 TL
Hisse Birim Fiyatı (USD): 198.05 TL
Hisse Birim Fiyatı (TL): 7056.23 TL

Hisse: AMZN
Tahmini Getiri (TL): 105.00 TL
Tahmini Götürü (TL): 97.00 TL
Hisse Birim Fiyatı (USD): 230.71 TL
Hisse Birim Fiyatı (TL): 8219.86 TL

Hisse: META
Tahmini Getiri (TL): 105.00 TL
Tahmini Götürü (TL): 97.00 TL
Hisse Birim Fiyatı (USD): 616.46 TL
Hisse Birim Fiyatı (TL): 21963.58 TL

Hisse: TSLA
Tahmini Getiri (TL): 105.00 TL
Tahmini Götürü (TL): 97.00 TL
Hisse Birim Fiyatı (USD): 424.07 TL
Hisse Birim Fiyatı (TL): 15109.00 TL

Hisse: BRK-B
Tahmini Getiri (TL): 105.00 TL
Tahmini Götürü (TL): 97.00 TL
Hisse Birim Fiyatı

In [8]:
import requests
import yfinance as yf

def get_stock_prices():
    """
    İlk 10 hisse senedinin kapanış fiyatlarını alır.
    """
    tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "BRK-B", "NVDA", "JPM", "V"]  # İlk 10 hisse
    stock_data = {}
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        stock_data[ticker] = stock.history(period="1d")['Close'].iloc[-1]  # Son kapanış fiyatı
    return stock_data

def calculate_stock_returns(amount, stock_prices, currency_rate, potential_increase=0.05, potential_decrease=0.03):
    """
    Her bir hisse için tahmini getiri ve götürüyü hesaplar.
    Args:
        amount (float): Yatırım miktarı (TL cinsinden).
        stock_prices (dict): Hisse fiyatları (USD cinsinden).
        currency_rate (float): Güncel USD/TRY kuru.
        potential_increase (float): Tahmini fiyat artış oranı (örneğin, %5 = 0.05).
        potential_decrease (float): Tahmini fiyat düşüş oranı (örneğin, %3 = 0.03).
    Returns:
        dict: Her hisse için tahmini getiri ve götürü, hem TL hem de USD cinsinden.
    """
    stock_results = {}
    for ticker, price in stock_prices.items():
        # Hisse başına alınabilecek birim sayısı
        units = amount / (price * currency_rate)
        
        # Tahmini getiri ve götürü hesaplama (USD bazında)
        expected_increase_usd = units * (price * (1 + potential_increase))
        expected_decrease_usd = units * (price * (1 - potential_decrease))
        
        # USD değerlerini TL'ye çevirme
        expected_increase_tl = expected_increase_usd * currency_rate
        expected_decrease_tl = expected_decrease_usd * currency_rate
        
        stock_results[ticker] = {
            "Tahmini Getiri (USD)": expected_increase_usd,
            "Tahmini Götürü (USD)": expected_decrease_usd,
            "Tahmini Getiri (TL)": expected_increase_tl,
            "Tahmini Götürü (TL)": expected_decrease_tl,
            "Hisse Birim Fiyatı (USD)": price,
            "Hisse Birim Fiyatı (TL)": price * currency_rate
        }
    
    return stock_results

def get_currency_rate():
    """
    Güncel USD/TRY kurunu alır.
    """
    url = "https://open.er-api.com/v6/latest/USD"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data["rates"]["TRY"]
    except requests.exceptions.RequestException as e:
        print("Döviz kuru alınamadı:", str(e))
        return None

def main():
    """
    Ana fonksiyon: Kullanıcıdan yatırım miktarını alır ve tahmini getiri/götürüyü hesaplar.
    """
    amount = float(input("Yatırım yapmak istediğiniz miktar (TL): "))
    
    # Güncel döviz kurunu al
    currency_rate = get_currency_rate()
    if not currency_rate:
        print("Döviz kuru alınamadı, program sonlandırılıyor.")
        return

    # Hisse fiyatlarını al
    stock_prices = get_stock_prices()
    
    # Her hisse için tahmini getiri ve götürüyü hesapla
    stock_results = calculate_stock_returns(amount, stock_prices, currency_rate)
    
    # Sonuçları ekrana yazdır
    if stock_results:
        print("\nTahmini Getiri ve Götürü (TL ve USD Bazında):")
        for ticker, data in stock_results.items():
            print(f"\nHisse: {ticker}")
            for key, value in data.items():
                print(f"{key}: {value:.2f}")

# Programı çalıştır
if __name__ == "__main__":
    main()



Tahmini Getiri ve Götürü (TL ve USD Bazında):

Hisse: AAPL
Tahmini Getiri (USD): 2.95
Tahmini Götürü (USD): 2.72
Tahmini Getiri (TL): 105.00
Tahmini Götürü (TL): 97.00
Hisse Birim Fiyatı (USD): 222.64
Hisse Birim Fiyatı (TL): 7932.34

Hisse: MSFT
Tahmini Getiri (USD): 2.95
Tahmini Götürü (USD): 2.72
Tahmini Getiri (TL): 105.00
Tahmini Götürü (TL): 97.00
Hisse Birim Fiyatı (USD): 428.50
Hisse Birim Fiyatı (TL): 15266.83

Hisse: GOOGL
Tahmini Getiri (USD): 2.95
Tahmini Götürü (USD): 2.72
Tahmini Getiri (TL): 105.00
Tahmini Götürü (TL): 97.00
Hisse Birim Fiyatı (USD): 198.05
Hisse Birim Fiyatı (TL): 7056.23

Hisse: AMZN
Tahmini Getiri (USD): 2.95
Tahmini Götürü (USD): 2.72
Tahmini Getiri (TL): 105.00
Tahmini Götürü (TL): 97.00
Hisse Birim Fiyatı (USD): 230.71
Hisse Birim Fiyatı (TL): 8219.86

Hisse: META
Tahmini Getiri (USD): 2.95
Tahmini Götürü (USD): 2.72
Tahmini Getiri (TL): 105.00
Tahmini Götürü (TL): 97.00
Hisse Birim Fiyatı (USD): 616.46
Hisse Birim Fiyatı (TL): 21963.58

Hisse: TS

# 1 Adım daha ilerisi (Yatırdığımız miktarın 1 hafta, 1 ay, 6 ay ve 1 yıl içerisindeki getiri ve götürüsü ne olurdu hesaplaması)

Hisselerin; 1 hafta, 1ay, 6ay ve 1yıllık fiyat değişimlerini internetten çekeceğiz ve tahmini getiri ve götürüyü bunlara göre gerekli formülleri kullanarak hesaplayacağız. Her hisse için getiri ve götürüyü hem TL hem de USD bazından göreceğiz. Eğer girilen miktar TL bazından ise USD'ye dönüşüm uygulayacağız

**Getiri/Götürü Hesaplama Formülü:**

Oran = $(BitisFiyati - BaslangıcFiyati) / BaslangıcFiyati$



In [9]:
import requests
import yfinance as yf

In [10]:
#Hisse geçmiş fiyatlarını çekme 
def get_stock_prices_history(ticker, period):
    """
    Belirli bir döneme ait hisse fiyat geçmişini alır.
    Args:
        ticker (str): Hisse sembolü.
        period (str): Fiyat geçmişi için dönem (örneğin, "7d", "1mo", "6mo", "1y").
    Returns:
        dict: Dönem için başlangıç ve bitiş fiyatları.
    """
    stock = yf.Ticker(ticker)
    history = stock.history(period=period)
    
    if history.empty:
        return None
    start_price = history["Close"].iloc[0]
    end_price = history["Close"].iloc[-1]
    return {"start_price": start_price, "end_price": end_price}    

In [11]:
# Örnek kullanım
ticker = "NVDA"
period = "6mo"  # 1 haftalık fiyat geçmişi
result = get_stock_prices_history(ticker, period)
print(f"{ticker} için {period} dönemi başlangıç ve bitiş fiyatları: {result}")

NVDA için 6mo dönemi başlangıç ve bitiş fiyatları: {'start_price': 123.52091979980469, 'end_price': 140.8300018310547}


In [12]:
#Döviz kuru çekme 
def get_currency_rate():
    """
    Güncel USD/TRY kurunu alır.
    Returns:
        float: USD/TRY kuru.
    """
    url = "https://open.er-api.com/v6/latest/USD"
    try: 
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data["rates"]["TRY"]
    except requests.exceptions.RequestException as e:
        print("Döviz kuru alınmadı:", str(e))
        return None
        

In [13]:
# Örnek kullanım
currency_rate = get_currency_rate()
if currency_rate:
    print(f"Güncel USD/TRY kuru: {currency_rate}")
else:
    print("Kur bilgisi alınamadı.")

Güncel USD/TRY kuru: 35.628548


In [14]:
def calculate_stock_returns_from_history(amount, currency, stock_prices, currency_rate):
    """
    Geçmiş fiyat değişim oranlarına göre tahmini getiri ve götürü hesaplar.
    Args:
        amount (float): Yatırım miktarı.
        currency (str): Para birimi ("TL" veya "USD").
        stock_prices (dict): Hisse fiyat geçmişleri.
        currency_rate (float): Güncel USD/TRY kuru.
    Returns:
        dict: Her hisse için tahmini getiri ve götürü.
    """
    if currency == "TL":
        # Eğer para birimi TL ise, USD'ye çevir
        amount_usd = amount / currency_rate
    else:
        # Eğer para birimi USD ise, doğrudan işlem yap
        amount_usd = amount

    stock_results = {}
    for ticker, data in stock_prices.items():
        if not data:
            stock_results[ticker] = "Veri bulunamadı"
            continue
        
        # Fiyat değişim oranını hesapla
        change_rate = (data["end_price"] - data["start_price"]) / data["start_price"]
        
        # Tahmini getiri ve götürü (USD bazında)
        expected_increase_usd = amount_usd * (1 + change_rate)
        expected_decrease_usd = amount_usd * (1 - change_rate)
        
        # USD değerlerini TL'ye çevirme (sadece görüntüleme için)
        expected_increase_tl = expected_increase_usd * currency_rate
        expected_decrease_tl = expected_decrease_usd * currency_rate
        
        stock_results[ticker] = {
            "Başlangıç Fiyatı (USD)": data["start_price"],
            "Bitiş Fiyatı (USD)": data["end_price"],
            "Değişim Oranı (%)": change_rate * 100,
            "Tahmini Getiri (USD)": expected_increase_usd,
            "Tahmini Götürü (USD)": expected_decrease_usd,
            "Tahmini Getiri (TL)": expected_increase_tl,
            "Tahmini Götürü (TL)": expected_decrease_tl
        }
    return stock_results

In [15]:
# Örnek kullanım
amount = 1000  # 1000 TL yatırılıyor
currency = "TL"
stock_prices = {
    "AAPL": {"start_price": 148.89, "end_price": 151.12},
    "MSFT": {"start_price": 299.65, "end_price": 305.12}
}
currency_rate = 36.75  # USD/TRY kuru
returns = calculate_stock_returns_from_history(amount, currency, stock_prices, currency_rate)
for ticker, data in returns.items():
    print(f"\nHisse: {ticker}")
    for key, value in data.items():
        print(f"{key}: {value:.2f}")


Hisse: AAPL
Başlangıç Fiyatı (USD): 148.89
Bitiş Fiyatı (USD): 151.12
Değişim Oranı (%): 1.50
Tahmini Getiri (USD): 27.62
Tahmini Götürü (USD): 26.80
Tahmini Getiri (TL): 1014.98
Tahmini Götürü (TL): 985.02

Hisse: MSFT
Başlangıç Fiyatı (USD): 299.65
Bitiş Fiyatı (USD): 305.12
Değişim Oranı (%): 1.83
Tahmini Getiri (USD): 27.71
Tahmini Götürü (USD): 26.71
Tahmini Getiri (TL): 1018.25
Tahmini Götürü (TL): 981.75


In [16]:
#Ana Fonksiyon
def main():
    """
    Ana fonksiyon: Kullanıcıdan yatırım miktarını ve para birimini alır, tahmini sonuçları hesaplar.
    """
    amount = float(input("Yatırım yapmak istediğiniz miktar: "))
    currency = input("Para biriminiz nedir? (TL/USD): ").strip().upper()
    
    if currency not in ["TL", "USD"]:
        print("Geçersiz para birimi girdiniz. Lütfen 'TL' veya 'USD' olarak giriniz.")
        return
    
    # Güncel döviz kurunu al
    currency_rate = get_currency_rate()
    if not currency_rate:
        print("Döviz kuru alınamadı, program sonlandırılıyor.")
        return

    # Hisse geçmişlerini al
    tickers = ["AAPL", "MSFT"]
    periods = ["7d"]
    stock_prices = {ticker: get_stock_prices_history(ticker, "7d") for ticker in tickers}
    
    # Tahmini getiri ve götürüyü hesapla
    stock_results = calculate_stock_returns_from_history(amount, currency, stock_prices, currency_rate)
    
    # Sonuçları ekrana yazdır
    for ticker, data in stock_results.items():
        print(f"\nHisse: {ticker}")
        for key, value in data.items():
            print(f"{key}: {value:.2f}")

# Programı çalıştır
if __name__ == "__main__":
    main()


Geçersiz para birimi girdiniz. Lütfen 'TL' veya 'USD' olarak giriniz.


In [17]:
#ESAS

In [18]:
import requests
import yfinance as yf

def get_stock_prices_history(ticker, period):
    """
    Belirli bir döneme ait hisse fiyat geçmişini alır.
    Args:
        ticker (str): Hisse sembolü.
        period (str): Fiyat geçmişi için dönem (örneğin, "7d", "1mo", "6mo", "1y").
    Returns:
        dict: Dönem için başlangıç ve bitiş fiyatları.
    """
    stock = yf.Ticker(ticker)
    history = stock.history(period=period)
    if history.empty:
        return None
    start_price = history['Close'].iloc[0]
    end_price = history['Close'].iloc[-1]
    return {"start_price": start_price, "end_price": end_price}

def calculate_stock_returns_from_history(amount, currency, stock_prices, currency_rate):
    """
    Geçmiş fiyat değişim oranlarına göre tahmini getiri ve götürü hesaplar.
    Args:
        amount (float): Yatırım miktarı.
        currency (str): Para birimi ("TL" veya "USD").
        stock_prices (dict): Hisse fiyat geçmişleri.
        currency_rate (float): Güncel USD/TRY kuru.
    Returns:
        dict: Her hisse için tahmini getiri ve götürü.
    """
    if currency == "TL":
        # Eğer para birimi TL ise, USD'ye çevir
        amount_usd = amount / currency_rate
    else:
        # Eğer para birimi USD ise, doğrudan işlem yap
        amount_usd = amount

    stock_results = {}
    for ticker, periods in stock_prices.items():
        stock_results[ticker] = {}
        for period, data in periods.items():
            if not data:
                stock_results[ticker][period] = "Veri bulunamadı"
                continue
            
            # Fiyat değişim oranını hesapla
            change_rate = (data["end_price"] - data["start_price"]) / data["start_price"]
            
            # Tahmini getiri ve götürü (USD bazında)
            expected_increase_usd = amount_usd * (1 + change_rate)
            expected_decrease_usd = amount_usd * (1 - change_rate)
            
            # USD değerlerini TL'ye çevirme (sadece görüntüleme için)
            expected_increase_tl = expected_increase_usd * currency_rate
            expected_decrease_tl = expected_decrease_usd * currency_rate
            
            stock_results[ticker][period] = {
                "Başlangıç Fiyatı (USD)": data["start_price"],
                "Bitiş Fiyatı (USD)": data["end_price"],
                "Değişim Oranı (%)": change_rate * 100,
                "Tahmini Getiri (USD)": expected_increase_usd,
                "Tahmini Götürü (USD)": expected_decrease_usd,
                "Tahmini Getiri (TL)": expected_increase_tl,
                "Tahmini Götürü (TL)": expected_decrease_tl
            }
    return stock_results

def get_currency_rate():
    """
    Güncel USD/TRY kurunu alır.
    """
    url = "https://open.er-api.com/v6/latest/USD"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data["rates"]["TRY"]
    except requests.exceptions.RequestException as e:
        print("Döviz kuru alınamadı:", str(e))
        return None

def main():
    """
    Ana fonksiyon: Kullanıcıdan yatırım miktarını ve para birimini alır, tahmini sonuçları hesaplar.
    """
    amount = float(input("Yatırım yapmak istediğiniz miktar: "))
    currency = input("Para biriminiz nedir? (TL/USD): ").strip().upper()
    
    if currency not in ["TL", "USD"]:
        print("Geçersiz para birimi girdiniz. Lütfen 'TL' veya 'USD' olarak giriniz.")
        return
    
    # Güncel döviz kurunu al
    currency_rate = get_currency_rate()
    if not currency_rate:
        print("Döviz kuru alınamadı, program sonlandırılıyor.")
        return

    # Hisse geçmişlerini al
    tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "BRK-B", "NVDA", "JPM", "V"]
    periods = ["7d", "1mo", "6mo", "1y"]
    stock_prices = {ticker: {period: get_stock_prices_history(ticker, period) for period in periods} for ticker in tickers}
    
    # Tahmini getiri ve götürüyü hesapla
    stock_results = calculate_stock_returns_from_history(amount, currency, stock_prices, currency_rate)
    
    # Sonuçları ekrana yazdır
    print("\nTahmini Getiri ve Götürü (Farklı Dönemler için):")
    for ticker, data in stock_results.items():
        print(f"\nHisse: {ticker}")
        for period, result in data.items():
            print(f"  Dönem: {period}")
            if isinstance(result, str):
                print(f"    {result}")
            else:
                for key, value in result.items():
                    print(f"    {key}: {value:.2f}")

# Programı çalıştır
if __name__ == "__main__":
    main()


Geçersiz para birimi girdiniz. Lütfen 'TL' veya 'USD' olarak giriniz.


# Düzeltme

Kullanıcı, yatırmak istediği miktarı TL veya USD olarak girecek, girdiği miktar hangi para birimi ise (TL ise USD'ye, USD ise TL'ye) diğerine dönüştürülecek ve girdiği miktarı 1hafta, 1ay, 3ay, 6ay ve 1yıl önce belirtilen hisseye yatırsaydı şu anda ne kadar olacağının çıktısını alacak. Bu kısmın olduğu kod bloğu bu başlık altında mevcut.

belirtilen periyotlarda değişim oranı negatifse götürüyü, pozitifse getiriyi hesaplıyor. Ayrıca, kullanıcı girişinin TL veya USD olmasına bağlı olarak çevirme işlemi yapılıyor ve sonuçlar her iki para biriminde de gösteriliyor.

In [19]:
import requests 
import yfinance as yf 

### Yalnızca Miktar ve Birim soruluyor

In [25]:
import requests
import yfinance as yf

def get_stock_prices_history(ticker, period):
    """
    Belirli bir döneme ait hisse fiyat geçmişini alır.
    Args:
        ticker (str): Hisse sembolü.
        period (str): Fiyat geçmişi için dönem (örneğin, "7d", "1mo", "6mo", "1y").
    Returns:
        dict: Dönem için başlangıç ve bitiş fiyatları.
    """
    stock = yf.Ticker(ticker)
    history = stock.history(period=period)
    if history.empty:
        return None
    start_price = history['Close'].iloc[0]
    end_price = history['Close'].iloc[-1]
    return {"start_price": start_price, "end_price": end_price}

def calculate_stock_returns(amount, currency, stock_prices, currency_rate):
    """
    Belirli dönemlere göre tahmini getiri ve götürü hesaplar.
    Args:
        amount (float): Yatırım miktarı.
        currency (str): Para birimi ("TL" veya "USD").
        stock_prices (dict): Hisse fiyat geçmişleri.
        currency_rate (float): Güncel USD/TRY kuru.
    Returns:
        dict: Her hisse için tahmini getiri ve götürü.
    """
    # Yatırım miktarını USD'ye çevirme
    if currency == "TL":
        amount_usd = amount / currency_rate
    else:
        amount_usd = amount

    # USD miktarını TL'ye çevirme
    amount_tl = amount_usd * currency_rate

    stock_results = {}
    for ticker, periods in stock_prices.items():
        stock_results[ticker] = {}
        for period, data in periods.items():
            if not data:
                stock_results[ticker][period] = "Veri bulunamadı"
                continue
            
            # Fiyat değişim oranını hesapla
            change_rate = (data["end_price"] - data["start_price"]) / data["start_price"]
            
            # Getiri veya götürüyü hesaplama
            if change_rate > 0:  # Pozitif değişim
                result_usd = amount_usd * (1 + change_rate)
            else:  # Negatif değişim
                result_usd = amount_usd * (1 + change_rate)  # Zaten kaybı içeriyor
            
            # USD sonucunu TL'ye çevir
            result_tl = result_usd * currency_rate
            
            stock_results[ticker][period] = {
                "Başlangıç Fiyatı (USD)": data["start_price"],
                "Bitiş Fiyatı (USD)": data["end_price"],
                "Değişim Oranı (%)": change_rate * 100,
                "Sonuç (USD)": result_usd,
                "Sonuç (TL)": result_tl
            }
    
    return stock_results, amount_usd, amount_tl

def get_currency_rate():
    """
    Güncel USD/TRY kurunu alır.
    Returns:
        float: USD/TRY kuru.
    """
    url = "https://open.er-api.com/v6/latest/USD"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data["rates"]["TRY"]
    except requests.exceptions.RequestException as e:
        print("Döviz kuru alınamadı:", str(e))
        return None

def main():
    """
    Ana fonksiyon: Kullanıcıdan yatırım miktarını ve para birimini alır, tahmini sonuçları hesaplar.
    """
    amount = float(input("Yatırım yapmak istediğiniz miktar: "))
    currency = input("Para biriminiz nedir? (TL/USD): ").strip().upper()
    
    if currency not in ["TL", "USD"]:
        print("Geçersiz para birimi girdiniz. Lütfen 'TL' veya 'USD' olarak giriniz.")
        return
    
    # Güncel döviz kurunu al
    currency_rate = get_currency_rate()
    if not currency_rate:
        print("Döviz kuru alınamadı, program sonlandırılıyor.")
        return

    # Hisse geçmişlerini al
    tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "BRK-B", "NVDA", "JPM", "V"]
    periods = ["7d", "1mo", "3mo","6mo", "1y"]
    stock_prices = {ticker: {period: get_stock_prices_history(ticker, period) for period in periods} for ticker in tickers}
    
    # Tahmini getiri ve götürüyü hesapla
    stock_results, amount_usd, amount_tl = calculate_stock_returns(amount, currency, stock_prices, currency_rate)
    
    # Sonuçları ekrana yazdır
    print("\nGirdiğiniz Bilgiler:")
    print(f"  Yatırım Miktarı: {amount:.2f} {currency}")
    print(f"  USD Karşılığı: {amount_usd:.2f} USD")
    print(f"  TL Karşılığı: {amount_tl:.2f} TL")
    
    print("\nTahmini Sonuçlar (Farklı Dönemler için):")
    for ticker, data in stock_results.items():
        print(f"\nHisse: {ticker}")
        for period, result in data.items():
            print(f"  Dönem: {period}")
            if isinstance(result, str):
                print(f"    {result}")
            else:
                for key, value in result.items():
                    print(f"    {key}: {value:.2f}")

# Programı çalıştır
if __name__ == "__main__":
    main()



Girdiğiniz Bilgiler:
  Yatırım Miktarı: 100.00 TL
  USD Karşılığı: 2.81 USD
  TL Karşılığı: 100.00 TL

Tahmini Sonuçlar (Farklı Dönemler için):

Hisse: AAPL
  Dönem: 7d
    Başlangıç Fiyatı (USD): 236.85
    Bitiş Fiyatı (USD): 222.64
    Değişim Oranı (%): -6.00
    Sonuç (USD): 2.64
    Sonuç (TL): 94.00
  Dönem: 1mo
    Başlangıç Fiyatı (USD): 255.27
    Bitiş Fiyatı (USD): 222.64
    Değişim Oranı (%): -12.78
    Sonuç (USD): 2.45
    Sonuç (TL): 87.22
  Dönem: 3mo
    Başlangıç Fiyatı (USD): 235.60
    Bitiş Fiyatı (USD): 222.64
    Değişim Oranı (%): -5.50
    Sonuç (USD): 2.65
    Sonuç (TL): 94.50
  Dönem: 6mo
    Başlangıç Fiyatı (USD): 223.46
    Bitiş Fiyatı (USD): 222.64
    Değişim Oranı (%): -0.36
    Sonuç (USD): 2.80
    Sonuç (TL): 99.64
  Dönem: 1y
    Başlangıç Fiyatı (USD): 192.94
    Bitiş Fiyatı (USD): 222.64
    Değişim Oranı (%): 15.39
    Sonuç (USD): 3.24
    Sonuç (TL): 115.39

Hisse: MSFT
  Dönem: 7d
    Başlangıç Fiyatı (USD): 418.95
    Bitiş Fiyatı (USD)

### Miktar, birim ve hissenin adı soruluyor (tarihler sonuçta görünmüyor)

In [26]:
def get_stock_prices_history(ticker, period):
    """
    Belirli bir döneme ait hisse fiyat geçmişini alır.
    Args:
        ticker (str): Hisse sembolü.
        period (str): Fiyat geçmişi için dönem (örneğin, "7d", "1mo", "6mo", "1y").
    Returns:
        dict: Dönem için başlangıç ve bitiş fiyatları.
    """
    stock = yf.Ticker(ticker)
    history = stock.history(period=period)
    if history.empty:
        return None
    start_price = history['Close'].iloc[0]
    end_price = history['Close'].iloc[-1]
    return {"start_price": start_price, "end_price": end_price}

def calculate_stock_returns(amount, currency, stock_prices, currency_rate):
    """
    Belirli dönemlere göre tahmini getiri ve götürü hesaplar.
    Args:
        amount (float): Yatırım miktarı.
        currency (str): Para birimi ("TL" veya "USD").
        stock_prices (dict): Hisse fiyat geçmişleri.
        currency_rate (float): Güncel USD/TRY kuru.
    Returns:
        dict: Her hisse için tahmini getiri ve götürü.
    """
    if currency == "TL":
        amount_usd = amount / currency_rate
    else:
        amount_usd = amount

    amount_tl = amount_usd * currency_rate

    stock_results = {}
    for ticker, periods in stock_prices.items():
        stock_results[ticker] = {}
        for period, data in periods.items():
            if not data:
                stock_results[ticker][period] = "Veri bulunamadı"
                continue

            change_rate = (data["end_price"] - data["start_price"]) / data["start_price"]

            result_usd = amount_usd * (1 + change_rate)
            result_tl = result_usd * currency_rate

            stock_results[ticker][period] = {
                "Başlangıç Fiyatı (USD)": data["start_price"],
                "Bitiş Fiyatı (USD)": data["end_price"],
                "Değişim Oranı (%)": change_rate * 100,
                "Sonuç (USD)": result_usd,
                "Sonuç (TL)": result_tl
            }

    return stock_results, amount_usd, amount_tl

def get_currency_rate():
    """
    Güncel USD/TRY kurunu alır.
    Returns:
        float: USD/TRY kuru.
    """
    url = "https://open.er-api.com/v6/latest/USD"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data["rates"]["TRY"]
    except requests.exceptions.RequestException as e:
        print("Döviz kuru alınamadı:", str(e))
        return None

def main():
    """
    Ana fonksiyon: Kullanıcıdan yatırım miktarını, para birimini ve hisse sembollerini alır, tahmini sonuçları hesaplar.
    """
    amount = float(input("Yatırım yapmak istediğiniz miktar: "))
    currency = input("Para biriminiz nedir? (TL/USD): ").strip().upper()

    if currency not in ["TL", "USD"]:
        print("Geçersiz para birimi girdiniz. Lütfen 'TL' veya 'USD' olarak giriniz.")
        return

    currency_rate = get_currency_rate()
    if not currency_rate:
        print("Döviz kuru alınamadı, program sonlandırılıyor.")
        return

    tickers = input("Hangi hisse senetlerine yatırım yapmak istiyorsunuz? (Virgülle ayırarak girin, örn: AAPL,MSFT): ").strip().upper().split(",")
    periods = ["7d", "1mo", "3mo", "6mo", "1y"]

    stock_prices = {ticker: {period: get_stock_prices_history(ticker, period) for period in periods} for ticker in tickers}

    stock_results, amount_usd, amount_tl = calculate_stock_returns(amount, currency, stock_prices, currency_rate)

    print("\nGirdiğiniz Bilgiler:")
    print(f"  Yatırım Miktarı: {amount:.2f} {currency}")
    print(f"  USD Karşılığı: {amount_usd:.2f} USD")
    print(f"  TL Karşılığı: {amount_tl:.2f} TL")

    print("\nTahmini Sonuçlar (Farklı Dönemler için):")
    for ticker, data in stock_results.items():
        print(f"\nHisse: {ticker}")
        for period, result in data.items():
            print(f"  Dönem: {period}")
            if isinstance(result, str):
                print(f"    {result}")
            else:
                for key, value in result.items():
                    print(f"    {key}: {value:.2f}")

if __name__ == "__main__":
    main()


Girdiğiniz Bilgiler:
  Yatırım Miktarı: 1000.00 USD
  USD Karşılığı: 1000.00 USD
  TL Karşılığı: 35628.55 TL

Tahmini Sonuçlar (Farklı Dönemler için):

Hisse: NVDA
  Dönem: 7d
    Başlangıç Fiyatı (USD): 135.91
    Bitiş Fiyatı (USD): 140.83
    Değişim Oranı (%): 3.62
    Sonuç (USD): 1036.20
    Sonuç (TL): 36918.32
  Dönem: 1mo
    Başlangıç Fiyatı (USD): 139.67
    Bitiş Fiyatı (USD): 140.83
    Değişim Oranı (%): 0.83
    Sonuç (USD): 1008.31
    Sonuç (TL): 35924.45
  Dönem: 3mo
    Başlangıç Fiyatı (USD): 143.58
    Bitiş Fiyatı (USD): 140.83
    Değişim Oranı (%): -1.92
    Sonuç (USD): 980.85
    Sonuç (TL): 34946.13
  Dönem: 6mo
    Başlangıç Fiyatı (USD): 123.52
    Bitiş Fiyatı (USD): 140.83
    Değişim Oranı (%): 14.01
    Sonuç (USD): 1140.13
    Sonuç (TL): 40621.20
  Dönem: 1y
    Başlangıç Fiyatı (USD): 59.64
    Bitiş Fiyatı (USD): 140.83
    Değişim Oranı (%): 136.14
    Sonuç (USD): 2361.45
    Sonuç (TL): 84135.02


### Miktar, birim ve hisse adı soruluyor (Tarihler sonuçta görünüyor)

In [1]:
import requests
import yfinance as yf
from datetime import datetime, timedelta

def get_stock_prices_history(ticker, period):
    """
    Belirli bir döneme ait hisse fiyat geçmişini alır.
    Args:
        ticker (str): Hisse sembolü.
        period (str): Fiyat geçmişi için dönem (örneğin, "7d", "1mo", "6mo", "1y").
    Returns:
        dict: Dönem için başlangıç ve bitiş fiyatları.
    """
    stock = yf.Ticker(ticker)
    history = stock.history(period=period)
    if history.empty:
        return None
    start_price = history['Close'].iloc[0]
    end_price = history['Close'].iloc[-1]
    return {"start_price": start_price, "end_price": end_price}

def calculate_stock_returns(amount, currency, stock_prices, currency_rate):
    """
    Belirli dönemlere göre tahmini getiri ve götürü hesaplar.
    Args:
        amount (float): Yatırım miktarı.
        currency (str): Para birimi ("TL" veya "USD").
        stock_prices (dict): Hisse fiyat geçmişleri.
        currency_rate (float): Güncel USD/TRY kuru.
    Returns:
        dict: Her hisse için tahmini getiri ve götürü.
    """
    if currency == "TL":
        amount_usd = amount / currency_rate
    else:
        amount_usd = amount

    amount_tl = amount_usd * currency_rate

    stock_results = {}
    for ticker, periods in stock_prices.items():
        stock_results[ticker] = {}
        for period, data in periods.items():
            if not data:
                stock_results[ticker][period] = "Veri bulunamadı"
                continue

            change_rate = (data["end_price"] - data["start_price"]) / data["start_price"]

            result_usd = amount_usd * (1 + change_rate)
            result_tl = result_usd * currency_rate

            stock_results[ticker][period] = {
                "Başlangıç Fiyatı (USD)": data["start_price"],
                "Bitiş Fiyatı (USD)": data["end_price"],
                "Değişim Oranı (%)": change_rate * 100,
                "Sonuç (USD)": result_usd,
                "Sonuç (TL)": result_tl
            }

    return stock_results, amount_usd, amount_tl

def get_period_date_range(period):
    """
    Belirli bir dönem için tarih aralığını hesaplar.
    Args:
        period (str): Dönem ("7d", "1mo", "3mo", "6mo", "1y").
    Returns:
        str: Başlangıç ve bitiş tarihini içeren bir string.
    """
    end_date = datetime.today()
    if period == "7d":
        start_date = end_date - timedelta(days=7)
    elif period == "1mo":
        start_date = end_date - timedelta(days=30)
    elif period == "3mo":
        start_date = end_date - timedelta(days=90)
    elif period == "6mo":
        start_date = end_date - timedelta(days=180)
    elif period == "1y":
        start_date = end_date - timedelta(days=365)
    else:
        start_date = None

    return f"{start_date.strftime('%d.%m.%Y')} - {end_date.strftime('%d.%m.%Y')}" if start_date else "Tarih bilgisi hesaplanamadı"

def get_currency_rate():
    """
    Güncel USD/TRY kurunu alır.
    Returns:
        float: USD/TRY kuru.
    """
    url = "https://open.er-api.com/v6/latest/USD"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data["rates"]["TRY"]
    except requests.exceptions.RequestException as e:
        print("Döviz kuru alınamadı:", str(e))
        return None

def main():
    """
    Ana fonksiyon: Kullanıcıdan yatırım miktarını, para birimini ve hisse sembollerini alır, tahmini sonuçları hesaplar.
    """
    amount = float(input("Yatırım yapmak istediğiniz miktar: "))
    currency = input("Para biriminiz nedir? (TL/USD): ").strip().upper()

    if currency not in ["TL", "USD"]:
        print("Geçersiz para birimi girdiniz. Lütfen 'TL' veya 'USD' olarak giriniz.")
        return

    currency_rate = get_currency_rate()
    if not currency_rate:
        print("Döviz kuru alınamadı, program sonlandırılıyor.")
        return

    tickers = input("Hangi hisse senetlerine yatırım yapmak istiyorsunuz? (Virgülle ayırarak girin, örn: AAPL,MSFT): ").strip().upper().split(",")
    periods = ["7d", "1mo", "3mo", "6mo", "1y"]

    stock_prices = {ticker: {period: get_stock_prices_history(ticker, period) for period in periods} for ticker in tickers}

    stock_results, amount_usd, amount_tl = calculate_stock_returns(amount, currency, stock_prices, currency_rate)

    print("\nGirdiğiniz Bilgiler:")
    print(f"  Yatırım Miktarı: {amount:.2f} {currency}")
    print(f"  USD Karşılığı: {amount_usd:.2f} USD")
    print(f"  TL Karşılığı: {amount_tl:.2f} TL")

    print("\nTahmini Sonuçlar (Farklı Dönemler için):")
    for ticker, data in stock_results.items():
        print(f"\nHisse: {ticker}")
        for period, result in data.items():
            date_range = get_period_date_range(period)
            print(f"  Dönem: {period} - {date_range}")
            if isinstance(result, str):
                print(f"    {result}")
            else:
                for key, value in result.items():
                    print(f"    {key}: {value:.2f}")

if __name__ == "__main__":
    main()



Girdiğiniz Bilgiler:
  Yatırım Miktarı: 100.00 USD
  USD Karşılığı: 100.00 USD
  TL Karşılığı: 3562.85 TL

Tahmini Sonuçlar (Farklı Dönemler için):

Hisse: NVDA
  Dönem: 7d - 15.01.2025 - 22.01.2025
    Başlangıç Fiyatı (USD): 133.23
    Bitiş Fiyatı (USD): 140.83
    Değişim Oranı (%): 5.70
    Sonuç (USD): 105.70
    Sonuç (TL): 3766.10
  Dönem: 1mo - 23.12.2024 - 22.01.2025
    Başlangıç Fiyatı (USD): 139.67
    Bitiş Fiyatı (USD): 147.37
    Değişim Oranı (%): 5.51
    Sonuç (USD): 105.51
    Sonuç (TL): 3759.27
  Dönem: 3mo - 24.10.2024 - 22.01.2025
    Başlangıç Fiyatı (USD): 143.58
    Bitiş Fiyatı (USD): 147.37
    Değişim Oranı (%): 2.64
    Sonuç (USD): 102.64
    Sonuç (TL): 3656.90
  Dönem: 6mo - 26.07.2024 - 22.01.2025
    Başlangıç Fiyatı (USD): 123.52
    Bitiş Fiyatı (USD): 147.37
    Değişim Oranı (%): 19.31
    Sonuç (USD): 119.31
    Sonuç (TL): 4250.76
  Dönem: 1y - 23.01.2024 - 22.01.2025
    Başlangıç Fiyatı (USD): 59.64
    Bitiş Fiyatı (USD): 147.37
    Değişim

# Uygulamaya aynı periyodun gelecek tahminini eklemeye çalışacağım (ML modellerini kullanacağım)

In [4]:
import pandas as pd 

## Veri çekme ve Veri seti olarak hazırlama 

In [21]:
import yfinance as yf
stock = yf.Ticker("NVDA")
df = stock.history(period="1y")

In [22]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-01-22 00:00:00-05:00,60.031981,60.313903,59.053258,59.637093,452955000,0.0,0.0
2024-01-23 00:00:00-05:00,59.553118,59.893022,58.568396,59.856033,294654000,0.0,0.0
2024-01-24 00:00:00-05:00,60.286910,62.831186,59.921012,61.344608,560271000,0.0,0.0
2024-01-25 00:00:00-05:00,62.332327,62.701226,60.832752,61.599537,482777000,0.0,0.0
2024-01-26 00:00:00-05:00,60.942723,61.765492,60.555836,61.013702,390309000,0.0,0.0
...,...,...,...,...,...,...,...
2025-01-14 00:00:00-05:00,136.050003,136.380005,130.050003,131.759995,195590500,0.0,0.0
2025-01-15 00:00:00-05:00,133.649994,136.449997,131.289993,136.240005,185217300,0.0,0.0
2025-01-16 00:00:00-05:00,138.639999,138.750000,133.490005,133.570007,209235600,0.0,0.0


+ **yfinance** ile belirtilen hisseye ait: `Date, Open, High, Low, Close, Volume, Dividends, Stock Splits` bilgilerini çekebiliyoruz. Bu bilgileri kullanarak feature engineering yapacağız ve yeni sütunlar elde edeceğiz 

In [31]:
import pandas as pd
#Veri setini hazırlama 
def prepare_time_series_data(ticker, start_date="5y"):
    """
    Belirtilen hisse için zaman serisi analizine uygun bir veri hazırlar.
    Args:
        ticker (str): Hisse sembolü.
        start_date (str): Veriyi çekmek için başlangıç tarihi (ör: "1y", "2y").
    Returns:
        pandas.DataFrame: Analize hazır zaman serisi verisi.
    """
    #Hisse verilerini çek 
    stock = yf.Ticker(ticker)
    df = stock.history(period=start_date)
    
    #Eğer veri boşşa uyarı ver
    if df.empty:
        raise ValueError(f"{ticker} için veri alınamadı. Lütfen geçerli bir hisse sembolü girin.")
    
    # Index sıfırlama
    df = df.reset_index()
    
    #Tarih sütunundaki saat bilgisini silme 
    df["Date"] = pd.to_datetime(df["Date"]).dt.date

    # Tarih sütununu datetime formatına çevir
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # Kapanış fiyatını kontrol et ve uygun formata dönüştür
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

    # Geçersiz değerleri kaldır
    df.dropna(inplace=True)
    
    ##FeatureEngineering


    # Veriyi kontrol et
    if df.empty:
        raise ValueError(f"{ticker} için geçerli bir veri bulunamadı. Veriler temizlenirken kayıplar yaşandı.")

    return df

In [32]:
data_1 = prepare_time_series_data("NVDA")
data_1

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2020-01-22,6.217933,6.313283,6.199012,6.226646,239240000,0.0,0.0
1,2020-01-23,6.266728,6.303573,6.177851,6.295109,244516000,0.0,0.0
2,2020-01-24,6.410624,6.460416,6.181584,6.235857,373512000,0.0,0.0
3,2020-01-27,5.931136,6.030968,5.781513,5.979932,470536000,0.0,0.0
4,2020-01-28,6.045903,6.205733,5.992626,6.173369,310976000,0.0,0.0
...,...,...,...,...,...,...,...,...
1252,2025-01-14,136.050003,136.380005,130.050003,131.759995,195590500,0.0,0.0
1253,2025-01-15,133.649994,136.449997,131.289993,136.240005,185217300,0.0,0.0
1254,2025-01-16,138.639999,138.750000,133.490005,133.570007,209235600,0.0,0.0
1255,2025-01-17,136.690002,138.500000,135.460007,137.710007,201188800,0.0,0.0


+ **prepare_time_series_data** adlı fonksiyon, tahmin yapılmak istenilen hissenin 5 sene önceden bugüne kadarki kapanış verilerini çekerek bir DataFrame oluşturuyor. Model kurmayı ilk olarak fonksiyon olmadan yapacağım. Öncesinde gerekli EDA işlemlerini ve Feature eng işlemlerini yapacağım 

In [33]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1257 entries, 0 to 1256
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          1257 non-null   datetime64[ns]
 1   Open          1257 non-null   float64       
 2   High          1257 non-null   float64       
 3   Low           1257 non-null   float64       
 4   Close         1257 non-null   float64       
 5   Volume        1257 non-null   int64         
 6   Dividends     1257 non-null   float64       
 7   Stock Splits  1257 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 78.7 KB


In [34]:
#Günlük fiyat değişimi
data_1["Daily_Change"] = data_1["Close"] - data_1["Open"]

In [35]:
data_1

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Daily_Change
0,2020-01-22,6.217933,6.313283,6.199012,6.226646,239240000,0.0,0.0,0.008713
1,2020-01-23,6.266728,6.303573,6.177851,6.295109,244516000,0.0,0.0,0.028381
2,2020-01-24,6.410624,6.460416,6.181584,6.235857,373512000,0.0,0.0,-0.174767
3,2020-01-27,5.931136,6.030968,5.781513,5.979932,470536000,0.0,0.0,0.048796
4,2020-01-28,6.045903,6.205733,5.992626,6.173369,310976000,0.0,0.0,0.127466
...,...,...,...,...,...,...,...,...,...
1252,2025-01-14,136.050003,136.380005,130.050003,131.759995,195590500,0.0,0.0,-4.290009
1253,2025-01-15,133.649994,136.449997,131.289993,136.240005,185217300,0.0,0.0,2.590012
1254,2025-01-16,138.639999,138.750000,133.490005,133.570007,209235600,0.0,0.0,-5.069992
1255,2025-01-17,136.690002,138.500000,135.460007,137.710007,201188800,0.0,0.0,1.020004


In [36]:
#Günlük yüzdelik değişim
data_1["Percentage_Change"] = ((data_1['Close'] - data_1['Open']) / data_1['Open']) * 100

In [37]:
data_1

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Daily_Change,Percentage_Change
0,2020-01-22,6.217933,6.313283,6.199012,6.226646,239240000,0.0,0.0,0.008713,0.140134
1,2020-01-23,6.266728,6.303573,6.177851,6.295109,244516000,0.0,0.0,0.028381,0.452878
2,2020-01-24,6.410624,6.460416,6.181584,6.235857,373512000,0.0,0.0,-0.174767,-2.726212
3,2020-01-27,5.931136,6.030968,5.781513,5.979932,470536000,0.0,0.0,0.048796,0.822704
4,2020-01-28,6.045903,6.205733,5.992626,6.173369,310976000,0.0,0.0,0.127466,2.108302
...,...,...,...,...,...,...,...,...,...,...
1252,2025-01-14,136.050003,136.380005,130.050003,131.759995,195590500,0.0,0.0,-4.290009,-3.153259
1253,2025-01-15,133.649994,136.449997,131.289993,136.240005,185217300,0.0,0.0,2.590012,1.937906
1254,2025-01-16,138.639999,138.750000,133.490005,133.570007,209235600,0.0,0.0,-5.069992,-3.656948
1255,2025-01-17,136.690002,138.500000,135.460007,137.710007,201188800,0.0,0.0,1.020004,0.746217


In [38]:
data_1["Dividends"].unique()

array([0.   , 0.004, 0.01 ])

In [39]:
data_1["Stock Splits"].unique()

array([ 0.,  4., 10.])

In [40]:
#MA7, MA30, MA60, MA180, MA365 (1Hafta, 1Ay, 3Ay, 6Ay, 1Yıl) hareketli ortalamalar (PACF)
data_1['MA_7'] = data_1['Close'].rolling(window=7).mean()
data_1['MA_30'] = data_1['Close'].rolling(window=30).mean()
data_1['MA_60'] = data_1['Close'].rolling(window=60).mean()
data_1['MA_180'] = data_1['Close'].rolling(window=180).mean()
data_1["MA_365"] = data_1["Close"].rolling(window=360).mean()

In [41]:
data_1

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Daily_Change,Percentage_Change,MA_7,MA_30,MA_60,MA_180,MA_365
0,2020-01-22,6.217933,6.313283,6.199012,6.226646,239240000,0.0,0.0,0.008713,0.140134,NaN,NaN,NaN,NaN,NaN
1,2020-01-23,6.266728,6.303573,6.177851,6.295109,244516000,0.0,0.0,0.028381,0.452878,NaN,NaN,NaN,NaN,NaN
2,2020-01-24,6.410624,6.460416,6.181584,6.235857,373512000,0.0,0.0,-0.174767,-2.726212,NaN,NaN,NaN,NaN,NaN
3,2020-01-27,5.931136,6.030968,5.781513,5.979932,470536000,0.0,0.0,0.048796,0.822704,NaN,NaN,NaN,NaN,NaN
4,2020-01-28,6.045903,6.205733,5.992626,6.173369,310976000,0.0,0.0,0.127466,2.108302,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,2025-01-14,136.050003,136.380005,130.050003,131.759995,195590500,0.0,0.0,-4.290009,-3.153259,139.292855,137.772374,139.707973,123.544556,91.267998
1253,2025-01-15,133.649994,136.449997,131.289993,136.240005,185217300,0.0,0.0,2.590012,1.937906,138.117142,137.705692,139.696631,123.814143,91.528290
1254,2025-01-16,138.639999,138.750000,133.490005,133.570007,209235600,0.0,0.0,-5.069992,-3.656948,135.851430,137.537344,139.622956,124.068775,91.781627
1255,2025-01-17,136.690002,138.500000,135.460007,137.710007,201188800,0.0,0.0,1.020004,0.746217,135.504288,137.452666,139.523121,124.353933,92.050720


In [42]:
#Boş değerleri veriden çıkararak 1 senelik veriyi kaybetmemek için
#Forward veya Backward Doldurma yapacağız
data_1["MA_365"] = data_1['MA_365'].fillna(method='ffill').fillna(method='bfill')
data_1["MA_180"] = data_1['MA_180'].fillna(method='ffill').fillna(method='bfill')
data_1["MA_60"] =data_1['MA_60'].fillna(method='ffill').fillna(method='bfill')
data_1["MA_30"] =data_1['MA_30'].fillna(method='ffill').fillna(method='bfill')
data_1["MA_7"] =data_1['MA_7'].fillna(method='ffill').fillna(method='bfill')


C:\Users\ogulc\AppData\Local\Temp\ipykernel_17728\507718537.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_1["MA_365"] = data_1['MA_365'].fillna(method='ffill').fillna(method='bfill')
C:\Users\ogulc\AppData\Local\Temp\ipykernel_17728\507718537.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_1["MA_180"] = data_1['MA_180'].fillna(method='ffill').fillna(method='bfill')
C:\Users\ogulc\AppData\Local\Temp\ipykernel_17728\507718537.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_1["MA_60"] =data_1['MA_60'].fillna(method='ffill').fillna(method='bfill')
C:\Users\ogulc\AppData\Local\Temp\ipykernel_17728\507718537.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future vers

In [43]:
data_1

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Daily_Change,Percentage_Change,MA_7,MA_30,MA_60,MA_180,MA_365
0,2020-01-22,6.217933,6.313283,6.199012,6.226646,239240000,0.0,0.0,0.008713,0.140134,6.163340,6.592991,6.393891,8.971762,11.561544
1,2020-01-23,6.266728,6.303573,6.177851,6.295109,244516000,0.0,0.0,0.028381,0.452878,6.163340,6.592991,6.393891,8.971762,11.561544
2,2020-01-24,6.410624,6.460416,6.181584,6.235857,373512000,0.0,0.0,-0.174767,-2.726212,6.163340,6.592991,6.393891,8.971762,11.561544
3,2020-01-27,5.931136,6.030968,5.781513,5.979932,470536000,0.0,0.0,0.048796,0.822704,6.163340,6.592991,6.393891,8.971762,11.561544
4,2020-01-28,6.045903,6.205733,5.992626,6.173369,310976000,0.0,0.0,0.127466,2.108302,6.163340,6.592991,6.393891,8.971762,11.561544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,2025-01-14,136.050003,136.380005,130.050003,131.759995,195590500,0.0,0.0,-4.290009,-3.153259,139.292855,137.772374,139.707973,123.544556,91.267998
1253,2025-01-15,133.649994,136.449997,131.289993,136.240005,185217300,0.0,0.0,2.590012,1.937906,138.117142,137.705692,139.696631,123.814143,91.528290
1254,2025-01-16,138.639999,138.750000,133.490005,133.570007,209235600,0.0,0.0,-5.069992,-3.656948,135.851430,137.537344,139.622956,124.068775,91.781627
1255,2025-01-17,136.690002,138.500000,135.460007,137.710007,201188800,0.0,0.0,1.020004,0.746217,135.504288,137.452666,139.523121,124.353933,92.050720


In [87]:
#Veri setini hazırlama 
def prepare_time_series_data(ticker, start_date="5y"):
    """
    Belirtilen hisse için zaman serisi analizine uygun bir veri hazırlar.
    Args:
        ticker (str): Hisse sembolü.
        start_date (str): Veriyi çekmek için başlangıç tarihi (ör: "1y", "2y").
    Returns:
        pandas.DataFrame: Analize hazır zaman serisi verisi.
    """
    #Hisse verilerini çek 
    stock = yf.Ticker(ticker)
    df = stock.history(period=start_date)
    
    #Eğer veri boşşa uyarı ver
    if df.empty:
        raise ValueError(f"{ticker} için veri alınamadı. Lütfen geçerli bir hisse sembolü girin.")
    
    # Index sıfırlama
    df = df.reset_index()
    
    #Tarih sütunundaki saat bilgisini silme 
    df["Date"] = pd.to_datetime(df["Date"]).dt.date

    # Tarih sütununu datetime formatına çevir
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # Kapanış fiyatını kontrol et ve uygun formata dönüştür
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

    # Geçersiz değerleri kaldır
    df.dropna(inplace=True)
    
    ###FeatureEngineering###
    
    #Günlük yüzdelik değişim
    df["Percentage_Change"] = ((df['Close'] - df['Open']) / df['Open']) * 100
    
    #Günlük hacim yüzdelik değişimi
    df['Volume_Percentage_Change'] = df['Volume'].pct_change() * 100
    df.dropna(inplace=True)

    #MA7, MA30, MA60, MA180, MA365 (1Hafta, 1Ay, 3Ay, 6Ay, 1Yıl) hareketli ortalamalar (PACF)
    df['MA_7'] = df['Close'].rolling(window=7).mean()
    df['MA_30'] = df['Close'].rolling(window=30).mean()
    df['MA_60'] = df['Close'].rolling(window=60).mean()
    df['MA_90'] = df['Close'].rolling(window=90).mean()
    df['MA_180'] = df['Close'].rolling(window=180).mean()
    df["MA_365"] = df["Close"].rolling(window=365).mean()
    
    df["MA_365"] = df['MA_365'].interpolate(method="linear")
    df["MA_180"] = df['MA_180'].interpolate(method="linear")
    df["MA_90"] =df['MA_90'].interpolate(method="linear")
    df["MA_60"] =df['MA_60'].interpolate(method="linear")
    df["MA_30"] =df['MA_30'].interpolate(method="linear")
    df["MA_7"] =df['MA_7'].interpolate(method="linear")
    
    #Üstel ağırlıklı ortalamalar
    df['EWMA_7'] = df['Close'].ewm(span=7, adjust=False).mean()
    df['EWMA_30'] = df['Close'].ewm(span=30, adjust=False).mean()
    df['EWMA_60'] = df['Close'].ewm(span=60, adjust=False).mean()
    df['EWMA_90'] = df['Close'].ewm(span=90, adjust=False).mean()
    df['EWMA_180'] = df['Close'].ewm(span=180, adjust=False).mean()
    df['EWMA_365'] = df['Close'].ewm(span=365, adjust=False).mean()
    
    df["EWMA_365"] = df['EWMA_365'].interpolate(method="linear")
    df["EWMA_180"] = df['EWMA_180'].interpolate(method="linear")
    df["EWMA_90"] =df['EWMA_90'].interpolate(method="linear")
    df["EWMA_60"] =df['EWMA_60'].interpolate(method="linear")
    df["EWMA_30"] =df['EWMA_30'].interpolate(method="linear")
    df["EWMA_7"] =df['EWMA_7'].interpolate(method="linear")
    
    #Gereksiz sütunların bırakılması 
    df = df.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"], 
            axis =1)
    
    
    # Veriyi kontrol et
    if df.empty:
        raise ValueError(f"{ticker} için geçerli bir veri bulunamadı. Veriler temizlenirken kayıplar yaşandı.")

    return df

In [88]:
data_2 = prepare_time_series_data("NVDA")

In [89]:
data_2

,Date,Close,Percentage_Change,Volume_Percentage_Change,MA_7,MA_30,MA_60,MA_90,MA_180,MA_365,EWMA_7,EWMA_30,EWMA_60,EWMA_90,EWMA_180,EWMA_365
1,2020-01-23,6.295109,0.452878,2.205317,NaN,NaN,NaN,NaN,NaN,NaN,6.295109,6.295109,6.295109,6.295109,6.295109,6.295109
2,2020-01-24,6.235857,-2.726212,52.755648,NaN,NaN,NaN,NaN,NaN,NaN,6.280296,6.291286,6.293166,6.293807,6.294454,6.294785
3,2020-01-27,5.979931,0.822704,25.976140,NaN,NaN,NaN,NaN,NaN,NaN,6.205205,6.271199,6.282896,6.286908,6.290979,6.293065
4,2020-01-28,6.173371,2.108302,-33.910264,NaN,NaN,NaN,NaN,NaN,NaN,6.197246,6.264887,6.279305,6.284413,6.289679,6.292410
5,2020-01-29,6.112874,-0.747805,-16.762708,NaN,NaN,NaN,NaN,NaN,NaN,6.176153,6.255080,6.273848,6.280643,6.287726,6.291429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,2025-01-14,131.759995,-3.153259,-4.500976,139.292855,137.772374,139.707973,133.566126,123.544556,90.629925,136.465964,137.843893,136.329148,133.211936,120.998384,98.094760
1253,2025-01-15,136.240005,1.937906,-5.303530,138.117142,137.705692,139.696631,133.888866,123.814143,90.881956,136.409475,137.740417,136.326225,133.278487,121.166800,98.303204
1254,2025-01-16,133.570007,-3.656948,12.967633,135.851430,137.537344,139.622956,134.230598,124.068775,91.125997,135.699608,137.471358,136.235857,133.284894,121.303852,98.495919
1255,2025-01-17,137.710007,0.746217,-3.845808,135.504288,137.452666,139.523121,134.577892,124.353933,91.380930,136.202207,137.486755,136.284190,133.382149,121.485136,98.710203


In [86]:
data_2.isnull().sum()

Date                          0
Close                         0
Percentage_Change             0
Volume_Percentage_Change      0
MA_7                          6
MA_30                        29
MA_60                        59
MA_90                        89
MA_180                      179
MA_365                      364
EWMA_7                        0
EWMA_30                       0
EWMA_60                       0
EWMA_90                       0
EWMA_180                      0
EWMA_365                      0
dtype: int64

In [81]:
data_2.columns

Index(['Date', 'Close', 'Percentage_Change', 'Volume_Percentage_Change',
       'MA_7', 'MA_30', 'MA_60', 'MA_90', 'MA_180', 'MA_365', 'EWMA_7',
       'EWMA_30', 'EWMA_60', 'EWMA_90', 'EWMA_180', 'EWMA_365'],
      dtype='object')

In [82]:
data_2.isnull().sum()

Date                        0
Close                       0
Percentage_Change           0
Volume_Percentage_Change    0
MA_7                        0
MA_30                       0
MA_60                       0
MA_90                       0
MA_180                      0
MA_365                      0
EWMA_7                      0
EWMA_30                     0
EWMA_60                     0
EWMA_90                     0
EWMA_180                    0
EWMA_365                    0
dtype: int64

### NA satırları doldurmak yerine direkt olarak daha fazla veri çekip 1 senelik NA değerlerini veri setinden çıkartan fonksiyon

In [5]:
#Veri setini hazırlama 
def prepare_time_series_data(ticker, start_date="7y"):
    """
    Belirtilen hisse için zaman serisi analizine uygun bir veri hazırlar.
    Args:
        ticker (str): Hisse sembolü.
        start_date (str): Veriyi çekmek için başlangıç tarihi (ör: "1y", "2y").
    Returns:
        pandas.DataFrame: Analize hazır zaman serisi verisi.
    """
    #Hisse verilerini çek 
    stock = yf.Ticker(ticker)
    df = stock.history(period=start_date)
    
    #Eğer veri boşşa uyarı ver
    if df.empty:
        raise ValueError(f"{ticker} için veri alınamadı. Lütfen geçerli bir hisse sembolü girin.")
    
    # Index sıfırlama
    df = df.reset_index()
    
    #Tarih sütunundaki saat bilgisini silme 
    df["Date"] = pd.to_datetime(df["Date"]).dt.date

    # Tarih sütununu datetime formatına çevir
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # Kapanış fiyatını kontrol et ve uygun formata dönüştür
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
    
    ###FeatureEngineering###
    
    #Günlük yüzdelik değişim
    df["Percentage_Change"] = ((df['Close'] - df['Open']) / df['Open']) * 100
    
    #Günlük hacim yüzdelik değişimi
    df['Volume_Percentage_Change'] = df['Volume'].pct_change() * 100
    df.dropna(inplace=True)

    #MA7, MA30, MA60, MA180, MA365 (1Hafta, 1Ay, 3Ay, 6Ay, 1Yıl) hareketli ortalamalar (PACF)
    df['MA_7'] = df['Close'].rolling(window=7).mean()
    df['MA_30'] = df['Close'].rolling(window=30).mean()
    df['MA_60'] = df['Close'].rolling(window=60).mean()
    df['MA_90'] = df['Close'].rolling(window=90).mean()
    df['MA_180'] = df['Close'].rolling(window=180).mean()
    df["MA_365"] = df["Close"].rolling(window=365).mean()
    
    #Üstel ağırlıklı ortalamalar
    df['EWMA_7'] = df['Close'].ewm(span=7, adjust=False).mean()
    df['EWMA_30'] = df['Close'].ewm(span=30, adjust=False).mean()
    df['EWMA_60'] = df['Close'].ewm(span=60, adjust=False).mean()
    df['EWMA_90'] = df['Close'].ewm(span=90, adjust=False).mean()
    df['EWMA_180'] = df['Close'].ewm(span=180, adjust=False).mean()
    df['EWMA_365'] = df['Close'].ewm(span=365, adjust=False).mean()
    
    #Gereksiz sütunların ve NA satırlarının bırakılması 
    df = df.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"], 
            axis =1)
    df.dropna(inplace=True)
    df = df.reset_index(drop=True)
    
    # Veriyi kontrol et
    if df.empty:
        raise ValueError(f"{ticker} için geçerli bir veri bulunamadı. Veriler temizlenirken kayıplar yaşandı.")

    return df

In [6]:
data_3 = prepare_time_series_data("NVDA")

In [113]:
data_3

,Date,Close,Percentage_Change,Volume_Percentage_Change,MA_7,MA_30,MA_60,MA_90,MA_180,MA_365,EWMA_7,EWMA_30,EWMA_60,EWMA_90,EWMA_180,EWMA_365
0,2019-07-05,3.982144,-0.105986,60.784314,4.040690,3.723596,4.048057,4.115755,4.095405,5.145550,3.997759,3.882634,3.950977,4.027895,4.331451,4.828332
1,2019-07-08,3.907089,-0.392829,-16.836127,4.033412,3.731928,4.033696,4.116345,4.083277,5.140067,3.975092,3.884211,3.949538,4.025240,4.326762,4.823298
2,2019-07-09,3.909574,1.157484,3.963759,4.012394,3.742139,4.019608,4.117236,4.071543,5.134802,3.958712,3.885848,3.948228,4.022698,4.322152,4.818305
3,2019-07-10,3.977919,0.628696,31.854401,3.997588,3.756134,4.007292,4.118282,4.060676,5.129687,3.963514,3.891788,3.949201,4.021714,4.318348,4.813712
4,2019-07-11,4.132503,2.012276,50.983463,3.997979,3.777756,3.999749,4.120955,4.052093,5.124523,4.005761,3.907318,3.955211,4.024149,4.316295,4.809990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390,2025-01-14,131.759995,-3.153259,-4.500976,139.292855,137.772374,139.707973,133.566126,123.544556,90.629925,136.465964,137.843893,136.329148,133.211936,120.998383,98.093311
1391,2025-01-15,136.240005,1.937906,-5.303530,138.117142,137.705692,139.696631,133.888866,123.814143,90.881956,136.409475,137.740417,136.326225,133.278487,121.166799,98.301763
1392,2025-01-16,133.570007,-3.656948,12.967633,135.851430,137.537344,139.622956,134.230598,124.068775,91.125997,135.699608,137.471358,136.235857,133.284894,121.303851,98.494486
1393,2025-01-17,137.710007,0.746217,-3.845808,135.504288,137.452666,139.523121,134.577892,124.353933,91.380930,136.202207,137.486755,136.284190,133.382149,121.485135,98.708778


In [114]:
data_3.describe()

,Date,Close,Percentage_Change,Volume_Percentage_Change,MA_7,MA_30,MA_60,MA_90,MA_180,MA_365,EWMA_7,EWMA_30,EWMA_60,EWMA_90,EWMA_180,EWMA_365
count,1395,1395.000000,1395.000000,1395.000000,1395.000000,1395.000000,1395.000000,1395.000000,1395.000000,1395.000000,1395.000000,1395.000000,1395.000000,1395.000000,1395.000000,1395.000000
mean,2022-04-10 15:44:30.967741952,36.282147,0.113304,4.796130,35.996635,34.894323,33.436727,32.005866,28.110771,22.282981,35.995359,34.891117,33.480528,32.150603,28.752371,24.004522
min,2019-07-05 00:00:00,3.697332,-8.943272,-67.240555,3.808388,3.723596,3.890678,3.983361,3.950788,4.661773,3.806965,3.882634,3.948228,4.021714,4.238745,4.644930
25%,2020-11-18 12:00:00,12.913389,-1.363458,-16.793126,12.950169,13.071807,13.177396,12.509743,10.176867,7.576323,12.946127,13.151723,12.916046,12.341125,10.727164,8.765622
50%,2022-04-08 00:00:00,19.519695,0.107353,-2.113748,19.418285,19.035987,18.491550,18.409942,17.933905,19.750383,19.449680,18.945110,18.669592,18.665211,18.403750,17.933401
75%,2023-08-29 12:00:00,44.136784,1.677382,18.690939,44.078851,44.177833,43.693109,39.464133,30.507665,23.689360,44.090452,43.829739,42.529688,39.790285,33.541644,27.543141
max,2025-01-21 00:00:00,149.429993,13.004996,380.859906,146.947013,141.896882,139.782342,134.941744,124.675092,91.642392,145.670149,140.284123,136.617538,133.545838,121.698890,98.938949
std,NaN,38.755492,2.610164,35.955921,38.462498,37.312393,35.627493,33.861127,28.724269,20.079702,38.441452,37.208718,35.512453,33.815592,29.193941,22.614473


In [115]:
data_4 = prepare_time_series_data(ticker="AAPL")

In [116]:
data_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1395 entries, 0 to 1394
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Date                      1395 non-null   datetime64[ns]
 1   Close                     1395 non-null   float64       
 2   Percentage_Change         1395 non-null   float64       
 3   Volume_Percentage_Change  1395 non-null   float64       
 4   MA_7                      1395 non-null   float64       
 5   MA_30                     1395 non-null   float64       
 6   MA_60                     1395 non-null   float64       
 7   MA_90                     1395 non-null   float64       
 8   MA_180                    1395 non-null   float64       
 9   MA_365                    1395 non-null   float64       
 10  EWMA_7                    1395 non-null   float64       
 11  EWMA_30                   1395 non-null   float64       
 12  EWMA_60             

In [117]:
print("##########HEAD##########\n",
      data_4.head(),"\n")

print("##########TAIL##########\n",
      data_4.tail(),"\n")

##########HEAD##########
         Date      Close  Percentage_Change  Volume_Percentage_Change  \
0 2019-07-05  49.164383           0.432746                 51.958282   
1 2019-07-08  48.150917          -0.393403                 46.758565   
2 2019-07-09  48.444611           1.024101                -18.787936   
3 2019-07-10  48.923653           0.683671                -13.027991   
4 2019-07-11  48.567383          -0.767300                 12.821630   

        MA_7      MA_30      MA_60      MA_90     MA_180     MA_365  \
0  48.503071  46.059047  46.949768  46.141474  44.167260  44.659411   
1  48.510636  46.222423  46.950449  46.210540  44.141092  44.677225   
2  48.562222  46.401125  46.962699  46.287451  44.117825  44.697661   
3  48.744833  46.601734  46.983253  46.364836  44.104053  44.721414   
4  48.751709  46.797288  46.996430  46.435917  44.083953  44.743934   

      EWMA_7    EWMA_30    EWMA_60    EWMA_90   EWMA_180   EWMA_365  
0  48.579382  47.118373  46.412486  45.91012

# 1-) Miktar, Birim ve Hisse Girdisini Alıp; Geçmişten Günümüze 'Ne Olurdu' Çıktısı 
# 2-) Belirtilen Hisseye Ait 7 Yıl Önceki Verileri Çeken Fonksiyon
# 3-) Model Eğitimi ve En İyi Sonuç Veren Modelde Hiperparametre Ayarlama

In [1]:
import pandas as pd 
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta

## 1

In [14]:
def get_stock_prices_history(ticker, period):
    """
    Belirli bir döneme ait hisse fiyat geçmişini alır.
    Args:
        ticker (str): Hisse sembolü.
        period (str): Fiyat geçmişi için dönem (örneğin, "7d", "1mo", "6mo", "1y").
    Returns:
        dict: Dönem için başlangıç ve bitiş fiyatları.
    """
    stock = yf.Ticker(ticker)
    history = stock.history(period=period)
    if history.empty:
        return None
    start_price = history['Close'].iloc[0]
    end_price = history['Close'].iloc[-1]
    return {"start_price": start_price, "end_price": end_price}

def calculate_stock_returns(amount, currency, stock_prices, currency_rate):
    """
    Belirli dönemlere göre tahmini getiri ve götürü hesaplar.
    Args:
        amount (float): Yatırım miktarı.
        currency (str): Para birimi ("TL" veya "USD").
        stock_prices (dict): Hisse fiyat geçmişleri.
        currency_rate (float): Güncel USD/TRY kuru.
    Returns:
        dict: Her hisse için tahmini getiri ve götürü.
    """
    if currency == "TL":
        amount_usd = amount / currency_rate
    else:
        amount_usd = amount

    amount_tl = amount_usd * currency_rate

    stock_results = {}
    for ticker, periods in stock_prices.items():
        stock_results[ticker] = {}
        for period, data in periods.items():
            if not data:
                stock_results[ticker][period] = "Veri bulunamadı"
                continue

            change_rate = (data["end_price"] - data["start_price"]) / data["start_price"]

            result_usd = amount_usd * (1 + change_rate)
            result_tl = result_usd * currency_rate

            stock_results[ticker][period] = {
                "Başlangıç Fiyatı (USD)": data["start_price"],
                "Bitiş Fiyatı (USD)": data["end_price"],
                "Değişim Oranı (%)": change_rate * 100,
                "Sonuç (USD)": result_usd,
                "Sonuç (TL)": result_tl
            }

    return stock_results, amount_usd, amount_tl

def get_period_date_range(period):
    """
    Belirli bir dönem için tarih aralığını hesaplar.
    Args:
        period (str): Dönem ("7d", "1mo", "3mo", "6mo", "1y").
    Returns:
        str: Başlangıç ve bitiş tarihini içeren bir string.
    """
    end_date = datetime.today()
    if period == "7d":
        start_date = end_date - timedelta(days=7)
    elif period == "1mo":
        start_date = end_date - timedelta(days=30)
    elif period == "3mo":
        start_date = end_date - timedelta(days=90)
    elif period == "6mo":
        start_date = end_date - timedelta(days=180)
    elif period == "1y":
        start_date = end_date - timedelta(days=365)
    else:
        start_date = None

    return f"{start_date.strftime('%d.%m.%Y')} - {end_date.strftime('%d.%m.%Y')}" if start_date else "Tarih bilgisi hesaplanamadı"

def get_currency_rate():
    """
    Güncel USD/TRY kurunu alır.
    Returns:
        float: USD/TRY kuru.
    """
    url = "https://open.er-api.com/v6/latest/USD"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data["rates"]["TRY"]
    except requests.exceptions.RequestException as e:
        print("Döviz kuru alınamadı:", str(e))
        return None

def main():
    """
    Ana fonksiyon: Kullanıcıdan yatırım miktarını, para birimini ve hisse sembollerini alır, tahmini sonuçları hesaplar.
    """
    amount = float(input("Yatırım yapmak istediğiniz miktar: "))
    currency = input("Para biriminiz nedir? (TL/USD): ").strip().upper()

    if currency not in ["TL", "USD"]:
        print("Geçersiz para birimi girdiniz. Lütfen 'TL' veya 'USD' olarak giriniz.")
        return

    currency_rate = get_currency_rate()
    if not currency_rate:
        print("Döviz kuru alınamadı, program sonlandırılıyor.")
        return

    tickers = input("Hangi hisse senetlerine yatırım yapmak istiyorsunuz? (Virgülle ayırarak girin, örn: AAPL,MSFT): ").strip().upper().split(",")
    periods = ["7d", "1mo", "3mo", "6mo", "1y"]

    stock_prices = {ticker: {period: get_stock_prices_history(ticker, period) for period in periods} for ticker in tickers}

    stock_results, amount_usd, amount_tl = calculate_stock_returns(amount, currency, stock_prices, currency_rate)

    print("\nGirdiğiniz Bilgiler:")
    print(f"  Yatırım Miktarı: {amount:.2f} {currency}")
    print(f"  USD Karşılığı: {amount_usd:.2f} USD")
    print(f"  TL Karşılığı: {amount_tl:.2f} TL")

    print("\nTahmini Sonuçlar (Farklı Dönemler için):")
    for ticker, data in stock_results.items():
        print(f"\nHisse: {ticker}")
        for period, result in data.items():
            date_range = get_period_date_range(period)
            print(f"  Dönem: {period} - {date_range}")
            if isinstance(result, str):
                print(f"    {result}")
            else:
                for key, value in result.items():
                    print(f"    {key}: {value:.2f}")

if __name__ == "__main__":
    main()



Girdiğiniz Bilgiler:
  Yatırım Miktarı: 3000.00 USD
  USD Karşılığı: 3000.00 USD
  TL Karşılığı: 106885.64 TL

Tahmini Sonuçlar (Farklı Dönemler için):

Hisse: NVDA
  Dönem: 7d - 15.01.2025 - 22.01.2025
    Başlangıç Fiyatı (USD): 133.23
    Bitiş Fiyatı (USD): 140.83
    Değişim Oranı (%): 5.70
    Sonuç (USD): 3171.13
    Sonuç (TL): 112982.86
  Dönem: 1mo - 23.12.2024 - 22.01.2025
    Başlangıç Fiyatı (USD): 139.67
    Bitiş Fiyatı (USD): 147.12
    Değişim Oranı (%): 5.33
    Sonuç (USD): 3159.91
    Sonuç (TL): 112583.11
  Dönem: 3mo - 24.10.2024 - 22.01.2025
    Başlangıç Fiyatı (USD): 143.58
    Bitiş Fiyatı (USD): 147.12
    Değişim Oranı (%): 2.47
    Sonuç (USD): 3073.96
    Sonuç (TL): 109520.85
  Dönem: 6mo - 26.07.2024 - 22.01.2025
    Başlangıç Fiyatı (USD): 123.52
    Bitiş Fiyatı (USD): 147.12
    Değişim Oranı (%): 19.11
    Sonuç (USD): 3573.16
    Sonuç (TL): 127306.50
  Dönem: 1y - 23.01.2024 - 22.01.2025
    Başlangıç Fiyatı (USD): 59.64
    Bitiş Fiyatı (USD): 14

## 2 

In [8]:
#Veri setini hazırlama 
def prepare_time_series_data(ticker, start_date="7y"):
    """
    Belirtilen hisse için zaman serisi analizine uygun bir veri hazırlar.
    Args:
        ticker (str): Hisse sembolü.
        start_date (str): Veriyi çekmek için başlangıç tarihi (ör: "1y", "2y").
    Returns:
        pandas.DataFrame: Analize hazır zaman serisi verisi.
    """
    #Hisse verilerini çek 
    stock = yf.Ticker(ticker)
    df = stock.history(period=start_date)
    
    #Eğer veri boşşa uyarı ver
    if df.empty:
        raise ValueError(f"{ticker} için veri alınamadı. Lütfen geçerli bir hisse sembolü girin.")
    
    # Index sıfırlama
    df = df.reset_index()
    
    #Tarih sütunundaki saat bilgisini silme 
    df["Date"] = pd.to_datetime(df["Date"]).dt.date

    # Tarih sütununu datetime formatına çevir
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # Kapanış fiyatını kontrol et ve uygun formata dönüştür
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
    
    ###FeatureEngineering###
    
    #Günlük yüzdelik değişim
    df["Percentage_Change"] = ((df['Close'] - df['Open']) / df['Open']) * 100
    
    #Günlük hacim yüzdelik değişimi
    df['Volume_Percentage_Change'] = df['Volume'].pct_change() * 100
    df.dropna(inplace=True)

    #MA7, MA30, MA60, MA180, MA365 (1Hafta, 1Ay, 3Ay, 6Ay, 1Yıl) hareketli ortalamalar (PACF)
    df['MA_7'] = df['Close'].rolling(window=7).mean()
    df['MA_30'] = df['Close'].rolling(window=30).mean()
    df['MA_60'] = df['Close'].rolling(window=60).mean()
    df['MA_90'] = df['Close'].rolling(window=90).mean()
    df['MA_180'] = df['Close'].rolling(window=180).mean()
    df["MA_365"] = df["Close"].rolling(window=365).mean()
    
    #Üstel ağırlıklı ortalamalar
    df['EWMA_7'] = df['Close'].ewm(span=7, adjust=False).mean()
    df['EWMA_30'] = df['Close'].ewm(span=30, adjust=False).mean()
    df['EWMA_60'] = df['Close'].ewm(span=60, adjust=False).mean()
    df['EWMA_90'] = df['Close'].ewm(span=90, adjust=False).mean()
    df['EWMA_180'] = df['Close'].ewm(span=180, adjust=False).mean()
    df['EWMA_365'] = df['Close'].ewm(span=365, adjust=False).mean()
    
    #Gereksiz sütunların ve NA satırlarının bırakılması 
    df = df.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"], 
            axis =1)
    df.dropna(inplace=True)
    df = df.reset_index(drop=True)
    
    # Veriyi kontrol et
    if df.empty:
        raise ValueError(f"{ticker} için geçerli bir veri bulunamadı. Veriler temizlenirken kayıplar yaşandı.")

    return df

In [10]:
data_5 = prepare_time_series_data(ticker="NVDA")

In [12]:
data_5

,Date,Close,Percentage_Change,Volume_Percentage_Change,MA_7,MA_30,MA_60,MA_90,MA_180,MA_365,EWMA_7,EWMA_30,EWMA_60,EWMA_90,EWMA_180,EWMA_365
0,2019-07-05,3.982144,-0.105986,60.784314,4.040690,3.723596,4.048057,4.115755,4.095405,5.145550,3.997759,3.882633,3.950977,4.027895,4.331451,4.828332
1,2019-07-08,3.907089,-0.392829,-16.836127,4.033411,3.731928,4.033696,4.116345,4.083277,5.140067,3.975091,3.884211,3.949538,4.025240,4.326762,4.823298
2,2019-07-09,3.909574,1.157484,3.963759,4.012393,3.742139,4.019608,4.117236,4.071543,5.134802,3.958712,3.885848,3.948228,4.022698,4.322152,4.818305
3,2019-07-10,3.977919,0.628696,31.854401,3.997588,3.756134,4.007292,4.118282,4.060676,5.129687,3.963514,3.891788,3.949201,4.021714,4.318348,4.813713
4,2019-07-11,4.132503,2.012276,50.983463,3.997979,3.777756,3.999749,4.120955,4.052093,5.124523,4.005761,3.907318,3.955211,4.024149,4.316295,4.809990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390,2025-01-14,131.759995,-3.153259,-4.500976,139.292855,137.772374,139.707973,133.566126,123.544556,90.629925,136.465964,137.843893,136.329148,133.211936,120.998383,98.093311
1391,2025-01-15,136.240005,1.937906,-5.303530,138.117142,137.705692,139.696631,133.888866,123.814143,90.881956,136.409475,137.740417,136.326225,133.278487,121.166799,98.301763
1392,2025-01-16,133.570007,-3.656948,12.967633,135.851430,137.537344,139.622956,134.230598,124.068775,91.125997,135.699608,137.471358,136.235857,133.284894,121.303851,98.494486
1393,2025-01-17,137.710007,0.746217,-3.845808,135.504288,137.452666,139.523121,134.577892,124.353933,91.380930,136.202207,137.486755,136.284190,133.382149,121.485134,98.708778


## 1 ve 2'nin Birleşimi

In [2]:
import requests
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Belirli bir döneme ait hisse fiyat geçmişini alır
def get_stock_prices_history(ticker, period):
    stock = yf.Ticker(ticker)
    history = stock.history(period=period)
    if history.empty:
        return None
    start_price = history['Close'].iloc[0]
    end_price = history['Close'].iloc[-1]
    return {"start_price": start_price, "end_price": end_price}

# Belirli dönemlere göre tahmini getiri ve götürü hesaplar
def calculate_stock_returns(amount, currency, stock_prices, currency_rate):
    if currency == "TL":
        amount_usd = amount / currency_rate
    else:
        amount_usd = amount

    amount_tl = amount_usd * currency_rate

    stock_results = {}
    for ticker, periods in stock_prices.items():
        stock_results[ticker] = {}
        for period, data in periods.items():
            if not data:
                stock_results[ticker][period] = "Veri bulunamadı"
                continue

            change_rate = (data["end_price"] - data["start_price"]) / data["start_price"]

            result_usd = amount_usd * (1 + change_rate)
            result_tl = result_usd * currency_rate

            stock_results[ticker][period] = {
                "Başlangıç Fiyatı (USD)": data["start_price"],
                "Bitiş Fiyatı (USD)": data["end_price"],
                "Değişim Oranı (%)": change_rate * 100,
                "Sonuç (USD)": result_usd,
                "Sonuç (TL)": result_tl
            }

    return stock_results, amount_usd, amount_tl

# Belirli bir dönem için tarih aralığını hesaplar
def get_period_date_range(period):
    end_date = datetime.today()
    if period == "7d":
        start_date = end_date - timedelta(days=7)
    elif period == "1mo":
        start_date = end_date - timedelta(days=30)
    elif period == "3mo":
        start_date = end_date - timedelta(days=90)
    elif period == "6mo":
        start_date = end_date - timedelta(days=180)
    elif period == "1y":
        start_date = end_date - timedelta(days=365)
    else:
        start_date = None

    return f"{start_date.strftime('%d.%m.%Y')} - {end_date.strftime('%d.%m.%Y')}" if start_date else "Tarih bilgisi hesaplanamadı"

# Güncel USD/TRY kurunu alır
def get_currency_rate():
    url = "https://open.er-api.com/v6/latest/USD"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data["rates"]["TRY"]
    except requests.exceptions.RequestException as e:
        print("Döviz kuru alınamadı:", str(e))
        return None

# Geleceğe yönelik tahminler için veri hazırlar
def prepare_time_series_data(ticker, start_date="7y"):
    stock = yf.Ticker(ticker)
    df = stock.history(period=start_date)
    if df.empty:
        raise ValueError(f"{ticker} için veri alınamadı. Lütfen geçerli bir hisse sembolü girin.")

    df = df.reset_index()
    df["Date"] = pd.to_datetime(df["Date"]).dt.date
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

    # Feature Engineering
    df["Percentage_Change"] = ((df['Close'] - df['Open']) / df['Open']) * 100
    df['Volume_Percentage_Change'] = df['Volume'].pct_change() * 100
    df.dropna(inplace=True)

    df['MA_7'] = df['Close'].rolling(window=7).mean()
    df['MA_30'] = df['Close'].rolling(window=30).mean()
    df['MA_60'] = df['Close'].rolling(window=60).mean()
    df['MA_90'] = df['Close'].rolling(window=90).mean()
    df['MA_180'] = df['Close'].rolling(window=180).mean()
    df["MA_365"] = df["Close"].rolling(window=365).mean()

    df['EWMA_7'] = df['Close'].ewm(span=7, adjust=False).mean()
    df['EWMA_30'] = df['Close'].ewm(span=30, adjust=False).mean()
    df['EWMA_60'] = df['Close'].ewm(span=60, adjust=False).mean()
    df['EWMA_90'] = df['Close'].ewm(span=90, adjust=False).mean()
    df['EWMA_180'] = df['Close'].ewm(span=180, adjust=False).mean()
    df['EWMA_365'] = df['Close'].ewm(span=365, adjust=False).mean()

    df = df.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"], axis=1)
    df.dropna(inplace=True)
    df = df.reset_index(drop=True)
    return df

def main():
    """
    Geçmişten günümüze yatırım analizi ve geleceğe yönelik veri hazırlığı.
    """
    amount = float(input("Yatırım yapmak istediğiniz miktar: "))
    currency = input("Para biriminiz nedir? (TL/USD): ").strip().upper()

    if currency not in ["TL", "USD"]:
        print("Geçersiz para birimi girdiniz. Lütfen 'TL' veya 'USD' olarak giriniz.")
        return

    currency_rate = get_currency_rate()
    if not currency_rate:
        print("Döviz kuru alınamadı, program sonlandırılıyor.")
        return

    ticker = input("Hangi hisse senedine yatırım yapmak istiyorsunuz? (örn: AAPL): ").strip().upper()

    # Gelecek tahminler için veri hazırla
    try:
        future_df = prepare_time_series_data(ticker)
        print("\nGeleceğe Yönelik Veriler:")
        print(future_df.tail())
    except ValueError as e:
        print(f"Hata: {e}")
        return

    # Geçmiş verileri analiz et
    periods = ["7d", "1mo", "3mo", "6mo", "1y"]
    stock_prices = {ticker: {period: get_stock_prices_history(ticker, period) for period in periods}}
    stock_results, amount_usd, amount_tl = calculate_stock_returns(amount, currency, stock_prices, currency_rate)

    # Kullanıcı bilgilerini göster
    print("\nGirdiğiniz Bilgiler:")
    print(f"  Yatırım Miktarı: {amount:.2f} {currency}")
    if currency == "TL":
        print(f"  USD Karşılığı: {amount_usd:.2f} USD (1 USD = {currency_rate:.2f} TL)")
    else:
        print(f"  TL Karşılığı: {amount_tl:.2f} TL (1 USD = {currency_rate:.2f} TL)")

    # Geçmişten günümüze yatırım analizi
    print("\nGeçmişten Günümüze Yatırım Analizi:")
    for ticker, data in stock_results.items():
        print(f"\nHisse: {ticker}")
        for period, result in data.items():
            date_range = get_period_date_range(period)
            print(f"  Dönem: {period} - {date_range}")
            if isinstance(result, str):
                print(f"    {result}")
            else:
                for key, value in result.items():
                    if isinstance(value, float):
                        print(f"    {key}: {value:.2f}")
                    else:
                        print(f"    {key}: {value}")

if __name__ == "__main__":
    main()



Geleceğe Yönelik Veriler:
           Date       Close  Percentage_Change  Volume_Percentage_Change  \
1390 2025-01-15  136.240005           1.937906                 -5.303530   
1391 2025-01-16  133.570007          -3.656948                 12.967633   
1392 2025-01-17  137.710007           0.746217                 -3.845808   
1393 2025-01-21  140.830002           1.200056                 -1.709737   
1394 2025-01-22  147.070007           1.665978                 19.347304   

            MA_7       MA_30       MA_60       MA_90      MA_180     MA_365  \
1390  138.117142  137.705692  139.696631  133.888866  123.814143  90.881956   
1391  135.851430  137.537344  139.622956  134.230598  124.068775  91.125997   
1392  135.504288  137.452666  139.523121  134.577892  124.353933  91.380930   
1393  135.607145  137.309333  139.477286  134.941744  124.675092  91.642392   
1394  137.201431  137.376333  139.602613  135.277056  125.015500  91.923011   

          EWMA_7     EWMA_30     EWMA_60 

## 3

### Model denemeleri

In [5]:
#Model seçimi için örnek bir veri setinin 'prepare_time_series_data()' fonksiyonu ile internetten çekilmesi
df_1 = prepare_time_series_data("NVDA")

In [6]:
print("##### HEAD #####", "\n")
print(df.head(5), "\n")
print("##### TAIL #####", "\n")
print(df.tail(5), "\n")
print("##### SHAPE #####", "\n")
print(df.shape, "\n")
print("##### INFO #####", "\n")
print(df.info(), "\n")
print("##### DESCRIBE #####", "\n")
print(df.describe())

##### HEAD ##### 

        Date     Close  Percentage_Change  Volume_Percentage_Change      MA_7  \
0 2019-07-08  3.907089          -0.392829                -16.836127  4.033411   
1 2019-07-09  3.909574           1.157484                  3.963759  4.012393   
2 2019-07-10  3.977919           0.628696                 31.854401  3.997588   
3 2019-07-11  4.132502           2.012276                 50.983463  3.997979   
4 2019-07-12  4.165557           0.125447                -25.809892  4.017080   

      MA_30     MA_60     MA_90    MA_180    MA_365    EWMA_7   EWMA_30  \
0  3.731928  4.033696  4.116345  4.083277  5.140067  3.975091  3.884211   
1  3.742139  4.019608  4.117236  4.071543  5.134802  3.958712  3.885847   
2  3.756134  4.007292  4.118282  4.060676  5.129687  3.963514  3.891788   
3  3.777756  3.999749  4.120955  4.052093  5.124523  4.005761  3.907318   
4  3.801366  3.991305  4.124066  4.043411  5.119211  4.045710  3.923978   

    EWMA_60   EWMA_90  EWMA_180  EWMA_365  

In [7]:
#Bağımlı ve Bağımsız değişkenlerin ayrılması 
X = df_1.drop(columns=["Close", "Date"], axis=1)
y = df_1["Close"]

In [8]:
#Bağımlı ve Bağımsız değişkenler için kontrol 
print("##### HEAD (X) #####", "\n")
print(X.head(5), "\n")
print("##### HEAD (y) #####", "\n")
print(y.head(5))

##### HEAD (X) ##### 

   Percentage_Change  Volume_Percentage_Change      MA_7     MA_30     MA_60  \
0          -0.392829                -16.836127  4.033411  3.731928  4.033696   
1           1.157484                  3.963759  4.012393  3.742139  4.019608   
2           0.628696                 31.854401  3.997588  3.756134  4.007292   
3           2.012276                 50.983463  3.997979  3.777755  3.999749   
4           0.125447                -25.809892  4.017080  3.801366  3.991305   

      MA_90    MA_180    MA_365    EWMA_7   EWMA_30   EWMA_60   EWMA_90  \
0  4.116345  4.083277  5.140067  3.975091  3.884211  3.949538  4.025217   
1  4.117236  4.071543  5.134802  3.958712  3.885847  3.948227  4.022675   
2  4.118282  4.060676  5.129687  3.963514  3.891788  3.949201  4.021692   
3  4.120955  4.052093  5.124523  4.005761  3.907318  3.955211  4.024127   
4  4.124066  4.043411  5.119211  4.045710  3.923978  3.962107  4.027235   

   EWMA_180  EWMA_365  
0  4.325429  4.812889

In [19]:
#Train ve Test setleri oluşturma
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42, shuffle = False)

In [22]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
import xgboost as xgb

from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error, mean_squared_error

In [18]:
regression_models = {"DecisionTree": DecisionTreeRegressor(),
                     "RandomForest": RandomForestRegressor(),
                     "GradientBoosting": GradientBoostingRegressor(),
                     "AdaBoost": AdaBoostRegressor(),
                     "KNN": KNeighborsRegressor(),
                     "LinearRegression": LinearRegression(),
                     "LassoRegression": Lasso(),
                     "RidgeRegression": Ridge(),
                     "LinearSVR": SVR(kernel="linear"),
                     "RbfSVR": SVR(kernel="rbf"),
                     "PolynomialSVR": SVR(kernel="poly"),
                     "XGBoost": xgb.XGBRegressor()}


In [25]:
model_names = []
train_score = []
test_score = []
mse_score = []
mae_score = []
r2score = []

for name, reg in regression_models.items():
    reg.fit(X_train, y_train)
    predictions = reg.predict(X_test)
    sc_train = reg.score(X_train, y_train) 
    sc_test = reg.score(X_test, y_test)
    sc_mse = mean_squared_error(y_test, predictions)
    sc_mae = mean_absolute_error(y_test, predictions)
    sc_r2 = r2_score(y_test, predictions)
    
    model_names.append(name)
    train_score.append(sc_train)
    test_score.append(sc_test)
    mse_score.append(sc_mse)
    mae_score.append(sc_mae)
    r2score.append(sc_r2)
    
    print(f"{name} Train Setindeki Doğruluk: {sc_train}")
    print(f"{name} Test Setindeki Doğruluk: {sc_test}")
    print(f"{name} MSE Skoru: {sc_mse}")
    print(f"{name} MAE Skoru: {sc_mae}")
    print(f"{name} R^2 Skoru: {sc_r2}\n")

DecisionTree Train Setindeki Doğruluk: 1.0
DecisionTree Test Setindeki Doğruluk: -3.968798437763115
DecisionTree MSE Skoru: 4350.658039632393
DecisionTree MAE Skoru: 58.955937017713275
DecisionTree R^2 Skoru: -3.968798437763115

RandomForest Train Setindeki Doğruluk: 0.9996239311028042
RandomForest Test Setindeki Doğruluk: -4.020964895039097
RandomForest MSE Skoru: 4396.334759988364
RandomForest MAE Skoru: 59.34251636736734
RandomForest R^2 Skoru: -4.020964895039097

GradientBoosting Train Setindeki Doğruluk: 0.9990568779705626
GradientBoosting Test Setindeki Doğruluk: -3.935900270426348
GradientBoosting MSE Skoru: 4321.852549128876
GradientBoosting MAE Skoru: 58.70831193006348
GradientBoosting R^2 Skoru: -3.935900270426348

AdaBoost Train Setindeki Doğruluk: 0.9950522407877196
AdaBoost Test Setindeki Doğruluk: -3.8677414941659594
AdaBoost MSE Skoru: 4262.173024667774
AdaBoost MAE Skoru: 58.21403524613409
AdaBoost R^2 Skoru: -3.8677414941659594

KNN Train Setindeki Doğruluk: 0.98896115

+ En iyi sonuçları: `LinearRegression`, `LassoRegression`, `RidgeRegression`, `LinearSVR` modellerinde elde ediyoruz. 

### XGBoost ve LightGBM Modelleri 

In [65]:
"""

from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

def train_and_forecast_lightgbm(df, periods):
    """
    LightGBM ile zaman serisi tahmini yapar.
    Args:
        df (pd.DataFrame): Zaman serisi veri seti (özellik mühendisliği yapılmış).
        periods (list): Tahmin yapılacak dönemler (ör: [7, 30, 90, 180, 365]).
    Returns:
        dict: Tahmin edilen değerler.
    """
    
    #Özellikler ve hedef değişkenler
    X = df.drop(columns=["Date", "Close"], axis=1)
    y = df["Close"]
    
    #Train-Test Ayrımı
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
    
    #StandardScaler()
    std_scaler = StandardScaler()
    X_train_std = std_scaler.fit_transform(X_train)
    X_test_std = std_scaler.transform(X_test)
    
    #Model eğitimi
    model = LGBMRegressor(
        objective='regression',
        metric='rmse',
        boosting_type='gbdt',
        learning_rate=0.05,
        num_leaves=31,
        max_depth=-1,
        n_estimators=1000,
        random_state=42)
    
    model.fit(
        X_train_std,
        y_train,
        eval_set=[(X_test_std, y_test)])
    
    return model

"""
    

### LinearRegression, LassoRegression, RidgeRegression, LinearSVR için HyperparameterTuning

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [46]:
#####LinearRegression için GridSearch#####
def grid_search_linear_regression(X_train, y_train):
    # Pipeline: Standartlaştırma + LinearRegression
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
    ])
    
    # Grid Search Parametreleri
    param_grid = {
        'regressor__fit_intercept': [True, False],
    }

    # Grid Search
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', refit='neg_mean_squared_error'
    )
    grid_search.fit(X_train, y_train)

    return grid_search


In [36]:
#####LassoRegression için GridSearch#####
def grid_search_lasso(X_train, y_train):
    scoring = {
        'MAE': make_scorer(mean_absolute_error, greater_is_better=False),
        'MSE': make_scorer(mean_squared_error, greater_is_better=False),
        'R2': make_scorer(r2_score)
    }

    param_grid = {
        'alpha': [0.01, 0.1, 1, 10, 100],
        'max_iter': [1000, 5000, 10000]
    }

    model = Lasso()
    grid_search = GridSearchCV(
        model, param_grid, cv=5, scoring=scoring, refit='R2'
    )
    grid_search.fit(X_train, y_train)

    return grid_search

In [37]:
#####RidgeRegression için GridSearch#####
def grid_search_ridge(X_train, y_train):
    scoring = {
        'MAE': make_scorer(mean_absolute_error, greater_is_better=False),
        'MSE': make_scorer(mean_squared_error, greater_is_better=False),
        'R2': make_scorer(r2_score)
    }

    param_grid = {
        'alpha': [0.01, 0.1, 1, 10, 100],
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sag']
    }

    model = Ridge()
    grid_search = GridSearchCV(
        model, param_grid, cv=5, scoring=scoring, refit='R2'
    )
    grid_search.fit(X_train, y_train)

    return grid_search

In [38]:
#####LinearSVR için GridSearch#####
def grid_search_linear_svr(X_train, y_train):
    scoring = {
        'MAE': make_scorer(mean_absolute_error, greater_is_better=False),
        'MSE': make_scorer(mean_squared_error, greater_is_better=False),
        'R2': make_scorer(r2_score)
    }

    param_grid = {
        'C': [0.1, 1, 10, 100],
        'epsilon': [0.01, 0.1, 1],
        'loss': ['epsilon_insensitive', 'squared_epsilon_insensitive'],
        'max_iter': [1000, 5000, 10000]
    }

    model = LinearSVR()
    grid_search = GridSearchCV(
        model, param_grid, cv=5, scoring=scoring, refit='R2'
    )
    grid_search.fit(X_train, y_train)

    return grid_search


In [47]:
# LinearRegression için Grid Search
grid_lr = grid_search_linear_regression(X_train, y_train)
print("LinearRegression En İyi Parametreler:", grid_lr.best_params_)
print("LinearRegression Negatif MSE Skoru:", grid_lr.best_score_)

# Lasso için Grid Search
grid_lasso = grid_search_lasso(X_train, y_train)
print("Lasso En İyi Parametreler:", grid_lasso.best_params_)
print("Lasso R2 Skoru:", grid_lasso.best_score_)
print("Lasso MAE ve MSE Skorları:", grid_lasso.cv_results_['mean_test_MAE'], grid_lasso.cv_results_['mean_test_MSE'])

# Ridge için Grid Search
grid_ridge = grid_search_ridge(X_train, y_train)
print("Ridge En İyi Parametreler:", grid_ridge.best_params_)
print("Ridge R2 Skoru:", grid_ridge.best_score_)
print("Ridge MAE ve MSE Skorları:", grid_ridge.cv_results_['mean_test_MAE'], grid_ridge.cv_results_['mean_test_MSE'])

# LinearSVR için Grid Search
grid_svr = grid_search_linear_svr(X_train, y_train)
print("LinearSVR En İyi Parametreler:", grid_svr.best_params_)
print("LinearSVR R2 Skoru:", grid_svr.best_score_)
print("LinearSVR MAE ve MSE Skorları:", grid_svr.cv_results_['mean_test_MAE'], grid_svr.cv_results_['mean_test_MSE'])

LinearRegression En İyi Parametreler: {'regressor__fit_intercept': True}
LinearRegression Negatif MSE Skoru: -0.10434197484858147


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.897e+02, tolerance: 1.063e+01
  model = cd_fast.enet_coordinate_descent(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.096e+02, tolerance: 1.439e+01
  model = cd_fast.enet_coordinate_descent(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.840e+02, toleranc

Lasso En İyi Parametreler: {'alpha': 0.01, 'max_iter': 10000}
Lasso R2 Skoru: 0.989323391797168
Lasso MAE ve MSE Skorları: [-0.45140967 -0.28797447 -0.24129324 -0.45039468 -0.4206184  -0.4206184
 -0.56958947 -0.55925787 -0.55925787 -1.65143658 -1.64352893 -1.64352893
 -9.4609156  -9.46025484 -9.46025484] [-4.32907928e-01 -1.74394581e-01 -1.30148434e-01 -4.53652452e-01
 -4.07067250e-01 -4.07067250e-01 -7.47362846e-01 -7.24556757e-01
 -7.24556757e-01 -6.63301511e+00 -6.61142258e+00 -6.61142258e+00
 -1.71537336e+02 -1.71525681e+02 -1.71525681e+02]


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter 

Ridge En İyi Parametreler: {'alpha': 0.01, 'solver': 'auto'}
Ridge R2 Skoru: 0.9908709979441941
Ridge MAE ve MSE Skorları: [-0.21372424 -0.21372424 -0.21372424 -0.212509   -0.4414022  -0.21896033
 -0.21896033 -0.21896033 -0.20679216 -0.44131733 -0.21647836 -0.21647836
 -0.21647836 -0.21049339 -0.44145407 -0.26976627 -0.26976627 -0.26976627
 -0.27122727 -0.44293581 -0.43390312 -0.43390312 -0.43390312 -0.43380285
 -0.46009926] [-0.10609827 -0.10609827 -0.10609827 -0.10038849 -0.42523847 -0.11103851
 -0.11103851 -0.11103851 -0.09412864 -0.42515373 -0.10720765 -0.10720765
 -0.10720765 -0.09766799 -0.42554913 -0.16866687 -0.16866687 -0.16866687
 -0.17120479 -0.42899223 -0.41860473 -0.41860473 -0.41860473 -0.41889363
 -0.46535376]


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


NameError: name 'LinearSVR' is not defined

### RidgeRegression ile Pipeline (3)

In [83]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import numpy as np
import pandas as pd

def train_and_forecast(df, periods):
    """
    Linear Regression ile geleceğe yönelik tahmin yapar.
    Args:
        df (pd.DataFrame): Zaman serisi veri seti (özellik mühendisliği yapılmış).
        periods (list): Tahmin yapılacak dönemler (ör: [7, 30, 90, 180, 365]).
    Returns:
        dict: Tahmin edilen değerler.
    """
    
    #Bağımlı ve Bağımsız değişkenler
    X = df.drop(columns=["Date", "Close"], axis=1)
    y = df["Close"]
    
    #Pipeline Tanımlama
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Ölçeklendirme
        ('regressor', Ridge())  # Ridge Regresyon Modeli
    ])
                         
    #Modelin Eğitimi
    pipeline.fit(X,y)
    
    # Geleceğe yönelik tahmin yapma
    last_row = df.iloc[-1][features].values.reshape(1, -1)  # Son veri noktası
    future_predictions = {}

    for period in periods:
        future_date = pd.to_datetime(df['Date'].iloc[-1]) + pd.Timedelta(days=period)
        prediction = pipeline.predict(last_row)[0]
        future_predictions[future_date.strftime('%Y-%m-%d')] = prediction

        # Yeni tahmini gecikmeli özellik olarak ekleme
        last_row = np.roll(last_row, -1)  # Gecikmeleri kaydırma
        last_row[0, -1] = prediction

    return future_predictions

In [84]:
ornek_1 = prepare_time_series_data("NVDA")
future_predictions = train_and_forecast(df, periods=[7, 30, 90, 180, 365])

# Tahmin sonuçlarını yazdır
print("Geleceğe Yönelik Tahminler:")
for date, value in future_predictions.items():
    print(f"Tarih: {date}, Tahmin: {value:.2f}")

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: X has 14 features, but StandardScaler is expecting 19 features as input.

# 1-2-3 Birleşimi 

In [2]:
import requests
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import numpy as np

# Belirli bir döneme ait hisse fiyat geçmişini alır
def get_stock_prices_history(ticker, period):
    stock = yf.Ticker(ticker)
    history = stock.history(period=period)
    if history.empty:
        return None
    start_price = history['Close'].iloc[0]
    end_price = history['Close'].iloc[-1]
    return {"start_price": start_price, "end_price": end_price}

# Belirli dönemlere göre tahmini getiri ve götürü hesaplar
def calculate_stock_returns(amount, currency, stock_prices, currency_rate):
    if currency == "TL":
        amount_usd = amount / currency_rate
    else:
        amount_usd = amount

    amount_tl = amount_usd * currency_rate

    stock_results = {}
    for ticker, periods in stock_prices.items():
        stock_results[ticker] = {}
        for period, data in periods.items():
            if not data:
                stock_results[ticker][period] = "Veri bulunamadı"
                continue

            change_rate = (data["end_price"] - data["start_price"]) / data["start_price"]

            result_usd = amount_usd * (1 + change_rate)
            result_tl = result_usd * currency_rate

            stock_results[ticker][period] = {
                "Başlangıç Fiyatı (USD)": data["start_price"],
                "Bitiş Fiyatı (USD)": data["end_price"],
                "Değişim Oranı (%)": change_rate * 100,
                "Sonuç (USD)": result_usd,
                "Sonuç (TL)": result_tl
            }

    return stock_results, amount_usd, amount_tl

# Belirli bir dönem için tarih aralığını hesaplar
def get_period_date_range(period):
    end_date = datetime.today()
    if period == "7d":
        start_date = end_date - timedelta(days=7)
    elif period == "1mo":
        start_date = end_date - timedelta(days=30)
    elif period == "3mo":
        start_date = end_date - timedelta(days=90)
    elif period == "6mo":
        start_date = end_date - timedelta(days=180)
    elif period == "1y":
        start_date = end_date - timedelta(days=365)
    else:
        start_date = None

    return f"{start_date.strftime('%d.%m.%Y')} - {end_date.strftime('%d.%m.%Y')}" if start_date else "Tarih bilgisi hesaplanamadı"

# Güncel USD/TRY kurunu alır
def get_currency_rate():
    url = "https://open.er-api.com/v6/latest/USD"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data["rates"]["TRY"]
    except requests.exceptions.RequestException as e:
        print("Döviz kuru alınamadı:", str(e))
        return None

# Geleceğe yönelik tahminler için veri hazırlar
def prepare_time_series_data(ticker, start_date="7y"):
    stock = yf.Ticker(ticker)
    df = stock.history(period=start_date)
    if df.empty:
        raise ValueError(f"{ticker} için veri alınamadı. Lütfen geçerli bir hisse sembolü girin.")

    df = df.reset_index()
    df["Date"] = pd.to_datetime(df["Date"]).dt.date
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

    # Feature Engineering
    df["Percentage_Change"] = ((df['Close'] - df['Open']) / df['Open']) * 100
    df['Volume_Percentage_Change'] = df['Volume'].pct_change() * 100
    df.dropna(inplace=True)

    df['MA_7'] = df['Close'].rolling(window=7).mean()
    df['MA_30'] = df['Close'].rolling(window=30).mean()
    df['MA_60'] = df['Close'].rolling(window=60).mean()
    df['MA_90'] = df['Close'].rolling(window=90).mean()
    df['MA_180'] = df['Close'].rolling(window=180).mean()
    df["MA_365"] = df["Close"].rolling(window=365).mean()

    df['EWMA_7'] = df['Close'].ewm(span=7, adjust=False).mean()
    df['EWMA_30'] = df['Close'].ewm(span=30, adjust=False).mean()
    df['EWMA_60'] = df['Close'].ewm(span=60, adjust=False).mean()
    df['EWMA_90'] = df['Close'].ewm(span=90, adjust=False).mean()
    df['EWMA_180'] = df['Close'].ewm(span=180, adjust=False).mean()
    df['EWMA_365'] = df['Close'].ewm(span=365, adjust=False).mean()

    df = df.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"], axis=1)
    df.dropna(inplace=True)
    df = df.reset_index(drop=True)
    return df

# Pipeline ile tahmin yapar (Başlangıç için RidgeRegression modeli kullanılıyor)
def train_and_forecast(df, periods):
    df.dropna(inplace=True)

    features = list(df.drop(columns=["Date", "Close"], axis=1).columns)
    target = 'Close'

    X = df[features]
    y = df[target]

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', Ridge())
    ])

    pipeline.fit(X, y)

    last_row = df.iloc[-1][features].values.reshape(1, -1)
    future_predictions = {}
    for period in periods:
        future_date = df['Date'].iloc[-1] + pd.Timedelta(days=period)
        prediction = pipeline.predict(last_row)[0]
        future_predictions[future_date.strftime('%Y-%m-%d')] = prediction
        last_row = np.roll(last_row, -1)
        last_row[0, -1] = prediction

    return future_predictions

# Main
def main():
    amount = float(input("Yatırım yapmak istediğiniz miktar: "))
    currency = input("Para biriminiz nedir? (TL/USD): ").strip().upper()

    if currency not in ["TL", "USD"]:
        print("Geçersiz para birimi girdiniz. Lütfen 'TL' veya 'USD' olarak giriniz.")
        return

    currency_rate = get_currency_rate()
    if not currency_rate:
        print("Döviz kuru alınamadı, program sonlandırılıyor.")
        return

    ticker = input("Hangi hisse senedine yatırım yapmak istiyorsunuz? (örn: AAPL): ").strip().upper()

    try:
        df = prepare_time_series_data(ticker)
        future_predictions = train_and_forecast(df, periods=[7, 30, 90, 180, 365])
    except ValueError as e:
        print(f"Hata: {e}")
        return

    periods = ["7d", "1mo", "3mo", "6mo", "1y"]
    stock_prices = {ticker: {period: get_stock_prices_history(ticker, period) for period in periods}}
    stock_results, amount_usd, amount_tl = calculate_stock_returns(amount, currency, stock_prices, currency_rate)

    print("\nGirdiğiniz Bilgiler:")
    print(f"  Yatırım Miktarı: {amount:.2f} {currency}")
    if currency == "TL":
        print(f"  USD Karşılığı: {amount_usd:.2f} USD (1 USD = {currency_rate:.2f} TL)")
    else:
        print(f"  TL Karşılığı: {amount_tl:.2f} TL (1 USD = {currency_rate:.2f} TL)")

    print("\nGeçmişten Günümüze Yatırım Analizi:")
    for ticker, data in stock_results.items():
        print(f"\nHisse: {ticker}")
        for period, result in data.items():
            date_range = get_period_date_range(period)
            print(f"  Dönem: {period} - {date_range}")
            if isinstance(result, str):
                print(f"    {result}")
            else:
                for key, value in result.items():
                    if isinstance(value, float):
                        print(f"    {key}: {value:.2f}")
                    else:
                        print(f"    {key}: {value}")

    print("\nGeleceğe Yönelik Tahminler:")
    for date, prediction in future_predictions.items():
        print(f"  Tarih: {date}, Tahmin: {prediction:.2f}")

if __name__ == "__main__":
    main()


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Girdiğiniz Bilgiler:
  Yatırım Miktarı: 1000.00 USD
  TL Karşılığı: 35681.53 TL (1 USD = 35.68 TL)

Geçmişten Günümüze Yatırım Analizi:

Hisse: NVDA
  Dönem: 7d - 17.01.2025 - 24.01.2025
    Başlangıç Fiyatı (USD): 131.76
    Bitiş Fiyatı (USD): 147.22
    Değişim Oranı (%): 11.73
    Sonuç (USD): 1117.33
    Sonuç (TL): 39868.21
  Dönem: 1mo - 25.12.2024 - 24.01.2025
    Başlangıç Fiyatı (USD): 140.22
    Bitiş Fiyatı (USD): 147.22
    Değişim Oranı (%): 4.99
    Sonuç (USD): 1049.92
    Sonuç (TL): 37462.81
  Dönem: 3mo - 26.10.2024 - 24.01.2025
    Başlangıç Fiyatı (USD): 140.40
    Bitiş Fiyatı (USD): 147.22
    Değişim Oranı (%): 4.86
    Sonuç (USD): 1048.57
    Sonuç (TL): 37414.69
  Dönem: 6mo - 28.07.2024 - 24.01.2025
    Başlangıç Fiyatı (USD): 114.23
    Bitiş Fiyatı (USD): 147.22
    Değişim Oranı (%): 28.88
    Sonuç (USD): 1288.78
    Sonuç (TL): 45985.53
  Dönem: 1y - 25.01.2024 - 24.01.2025
    Başlangıç Fiyatı (USD): 61.34
    Bitiş Fiyatı (USD): 147.22
    Değişim Or

# Son Hal 

## TL girildiğinde USD karşılığı yerine TL karşılığı yazıyor (Hatalı çıktıya sahip)

In [11]:
import requests
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import numpy as np

# Belirli bir döneme ait hisse fiyat geçmişini alır
def get_stock_prices_history(ticker, period):
    stock = yf.Ticker(ticker)
    history = stock.history(period=period)
    if history.empty:
        return None
    start_price = history['Close'].iloc[0]
    end_price = history['Close'].iloc[-1]
    return {"start_price": start_price, "end_price": end_price}

# Belirli dönemlere göre tahmini getiri ve götürü hesaplar
def calculate_stock_returns(amount, currency, stock_prices, currency_rate):
    if currency == "TL":
        amount_usd = amount / currency_rate
    else:
        amount_usd = amount

    amount_tl = amount_usd * currency_rate

    stock_results = {}
    for ticker, periods in stock_prices.items():
        stock_results[ticker] = {}
        for period, data in periods.items():
            if not data:
                stock_results[ticker][period] = "Veri bulunamadı"
                continue

            change_rate = (data["end_price"] - data["start_price"]) / data["start_price"]

            result_usd = amount_usd * (1 + change_rate)
            result_tl = result_usd * currency_rate

            stock_results[ticker][period] = {
                "Başlangıç Fiyatı (USD)": data["start_price"],
                "Bitiş Fiyatı (USD)": data["end_price"],
                "Değişim Oranı (%)": change_rate * 100,
                "Sonuç (USD)": result_usd,
                "Sonuç (TL)": result_tl
            }

    return stock_results, amount_usd, amount_tl

# Belirli bir dönem için tarih aralığını hesaplar
def get_period_date_range(period):
    end_date = datetime.today()
    if period == "7d":
        start_date = end_date - timedelta(days=7)
    elif period == "1mo":
        start_date = end_date - timedelta(days=30)
    elif period == "3mo":
        start_date = end_date - timedelta(days=90)
    elif period == "6mo":
        start_date = end_date - timedelta(days=180)
    elif period == "1y":
        start_date = end_date - timedelta(days=365)
    else:
        start_date = None

    return f"{start_date.strftime('%d.%m.%Y')} - {end_date.strftime('%d.%m.%Y')}" if start_date else "Tarih bilgisi hesaplanamadı"

# Güncel USD/TRY kurunu alır
def get_currency_rate():
    url = "https://open.er-api.com/v6/latest/USD"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data["rates"]["TRY"]
    except requests.exceptions.RequestException as e:
        print("Döviz kuru alınamadı:", str(e))
        return None

# Geleceğe yönelik tahminler için veri hazırlar
def prepare_time_series_data(ticker, start_date="7y"):
    stock = yf.Ticker(ticker)
    df = stock.history(period=start_date)
    if df.empty:
        raise ValueError(f"{ticker} için veri alınamadı. Lütfen geçerli bir hisse sembolü girin.")

    df = df.reset_index()
    df["Date"] = pd.to_datetime(df["Date"]).dt.date
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

    # Feature Engineering
    df["Percentage_Change"] = ((df['Close'] - df['Open']) / df['Open']) * 100
    df['Volume_Percentage_Change'] = df['Volume'].pct_change() * 100
    df.dropna(inplace=True)

    df['MA_7'] = df['Close'].rolling(window=7).mean()
    df['MA_30'] = df['Close'].rolling(window=30).mean()
    df['MA_60'] = df['Close'].rolling(window=60).mean()
    df['MA_90'] = df['Close'].rolling(window=90).mean()
    df['MA_180'] = df['Close'].rolling(window=180).mean()
    df["MA_365"] = df["Close"].rolling(window=365).mean()

    df['EWMA_7'] = df['Close'].ewm(span=7, adjust=False).mean()
    df['EWMA_30'] = df['Close'].ewm(span=30, adjust=False).mean()
    df['EWMA_60'] = df['Close'].ewm(span=60, adjust=False).mean()
    df['EWMA_90'] = df['Close'].ewm(span=90, adjust=False).mean()
    df['EWMA_180'] = df['Close'].ewm(span=180, adjust=False).mean()
    df['EWMA_365'] = df['Close'].ewm(span=365, adjust=False).mean()

    df = df.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"], axis=1)
    df.dropna(inplace=True)
    df = df.reset_index(drop=True)
    return df

#Pipeline ile Tahmin
def train_and_forecast(df, periods, investment_amount_usd, currency_rate):
    df.dropna(inplace=True)

    features = list(df.drop(columns=["Date", "Close"], axis=1).columns)
    target = 'Close'

    X = df[features]
    y = df[target]

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', Ridge())
    ])

    pipeline.fit(X, y)

    last_row = df.iloc[-1][features].values.reshape(1, -1)
    future_predictions = {}
    for period in periods:
        future_date = df['Date'].iloc[-1] + pd.Timedelta(days=period)
        predicted_price = pipeline.predict(last_row)[0]
        future_investment_usd = investment_amount_usd * (predicted_price / df['Close'].iloc[-1])
        future_investment_tl = future_investment_usd * currency_rate

        future_predictions[future_date.strftime('%Y-%m-%d')] = {
            "Tahmini Fiyat (USD)": predicted_price,
            "Yatırım Sonucu (USD)": future_investment_usd,
            "Yatırım Sonucu (TL)": future_investment_tl
        }
        last_row = np.roll(last_row, -1)
        last_row[0, -1] = predicted_price

    return future_predictions

# Main
def main():
    amount = float(input("Yatırım yapmak istediğiniz miktar: "))
    currency = input("Para biriminiz nedir? (TL/USD): ").strip().upper()

    if currency not in ["TL", "USD"]:
        print("Geçersiz para birimi girdiniz. Lütfen 'TL' veya 'USD' olarak giriniz.")
        return

    currency_rate = get_currency_rate()
    if not currency_rate:
        print("Döviz kuru alınamadı, program sonlandırılıyor.")
        return

    if currency == "TL":
        investment_amount_usd = amount / currency_rate
    else:
        investment_amount_usd = amount

    investment_amount_tl = investment_amount_usd * currency_rate

    ticker = input("Hangi hisse senedine yatırım yapmak istiyorsunuz? (örn: AAPL): ").strip().upper()

    try:
        df = prepare_time_series_data(ticker)
        future_predictions = train_and_forecast(
            df, 
            periods=[7, 30, 90, 180, 365], 
            investment_amount_usd=investment_amount_usd, 
            currency_rate=currency_rate
        )
    except ValueError as e:
        print(f"Hata: {e}")
        return

    periods = ["7d", "1mo", "3mo", "6mo", "1y"]
    stock_prices = {ticker: {period: get_stock_prices_history(ticker, period) for period in periods}}
    stock_results, amount_usd, amount_tl = calculate_stock_returns(amount, currency, stock_prices, currency_rate)

    # Çıktı Birleştirme
    print("\nYatırım Analiz ve Tahmin Raporu")

    print("\nGirdiğiniz Bilgiler:")
    print(f"  Yatırım Miktarı: {amount:.2f} {currency}")
    print(f"  TL Karşılığı: {investment_amount_tl:.2f} TL (1 USD = {currency_rate:.2f} TL)")

    print("\nGeçmişten Günümüze Yatırım Analizi:")
    for ticker, data in stock_results.items():
        print(f"\nHisse: {ticker}")
        for period, result in data.items():
            date_range = get_period_date_range(period)
            print(f"  Dönem: {period} - {date_range}")
            if isinstance(result, str):
                print(f"    {result}")
            else:
                for key, value in result.items():
                    if isinstance(value, float):
                        print(f"    {key}: {value:.2f}")
                    else:
                        print(f"    {key}: {value}")

    print("\nGeleceğe Yönelik Tahminler:")
    for date, prediction in future_predictions.items():
        print(f"  Tarih: {date}")
        for key, value in prediction.items():
            if isinstance(value, float):
                print(f"    {key}: {value:.2f}")
            else:
                print(f"    {key}: {value}")

if __name__ == "__main__":
    main()


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names




Yatırım Analiz ve Tahmin Raporu

Girdiğiniz Bilgiler:
  Yatırım Miktarı: 1000.00 TL
  TL Karşılığı: 1000.00 TL (1 USD = 35.68 TL)

Geçmişten Günümüze Yatırım Analizi:

Hisse: NVDA
  Dönem: 7d - 17.01.2025 - 24.01.2025
    Başlangıç Fiyatı (USD): 131.76
    Bitiş Fiyatı (USD): 147.22
    Değişim Oranı (%): 11.73
    Sonuç (USD): 31.31
    Sonuç (TL): 1117.33
  Dönem: 1mo - 25.12.2024 - 24.01.2025
    Başlangıç Fiyatı (USD): 140.22
    Bitiş Fiyatı (USD): 147.22
    Değişim Oranı (%): 4.99
    Sonuç (USD): 29.42
    Sonuç (TL): 1049.92
  Dönem: 3mo - 26.10.2024 - 24.01.2025
    Başlangıç Fiyatı (USD): 140.40
    Bitiş Fiyatı (USD): 147.22
    Değişim Oranı (%): 4.86
    Sonuç (USD): 29.39
    Sonuç (TL): 1048.57
  Dönem: 6mo - 28.07.2024 - 24.01.2025
    Başlangıç Fiyatı (USD): 114.23
    Bitiş Fiyatı (USD): 147.22
    Değişim Oranı (%): 28.88
    Sonuç (USD): 36.12
    Sonuç (TL): 1288.78
  Dönem: 1y - 25.01.2024 - 24.01.2025
    Başlangıç Fiyatı (USD): 61.34
    Bitiş Fiyatı (USD): 14

## TL Girildiğinde USD, USD girildiğinde TL karşılığı yazıyor (Hata düzeltildi)

In [20]:
import requests
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import numpy as np

#Belirli bir döneme ait hisse fiyat geçmişini alma
def get_stock_prices_history(ticker, period):
    stock = yf.Ticker(ticker)
    history = stock.history(period=period)
    if history.empty:
        return None
    start_price = history['Close'].iloc[0]
    end_price = history['Close'].iloc[-1]
    return {"start_price": start_price, "end_price": end_price}

#Belirli dönemlere göre tahmini getiri ve götürü hesaplama
def calculate_stock_returns(amount, currency, stock_prices, currency_rate):
    if currency == "TL":
        amount_usd = amount / currency_rate
    else:
        amount_usd = amount

    amount_tl = amount_usd * currency_rate

    stock_results = {}
    for ticker, periods in stock_prices.items():
        stock_results[ticker] = {}
        for period, data in periods.items():
            if not data:
                stock_results[ticker][period] = "Veri bulunamadı"
                continue

            change_rate = (data["end_price"] - data["start_price"]) / data["start_price"]

            result_usd = amount_usd * (1 + change_rate)
            result_tl = result_usd * currency_rate

            stock_results[ticker][period] = {
                "Başlangıç Fiyatı (USD)": data["start_price"],
                "Bitiş Fiyatı (USD)": data["end_price"],
                "Değişim Oranı (%)": change_rate * 100,
                "Sonuç (USD)": result_usd,
                "Sonuç (TL)": result_tl
            }

    return stock_results, amount_usd, amount_tl

#Belirli bir dönem için tarih aralığını hesaplama
def get_period_date_range(period):
    end_date = datetime.today()
    if period == "7d":
        start_date = end_date - timedelta(days=7)
    elif period == "1mo":
        start_date = end_date - timedelta(days=30)
    elif period == "3mo":
        start_date = end_date - timedelta(days=90)
    elif period == "6mo":
        start_date = end_date - timedelta(days=180)
    elif period == "1y":
        start_date = end_date - timedelta(days=365)
    else:
        start_date = None

    return f"{start_date.strftime('%d.%m.%Y')} - {end_date.strftime('%d.%m.%Y')}" if start_date else "Tarih bilgisi hesaplanamadı"

#Güncel USD/TRY kurunu alma
def get_currency_rate():
    url = "https://open.er-api.com/v6/latest/USD"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return data["rates"]["TRY"]
    except requests.exceptions.RequestException as e:
        print("Döviz kuru alınamadı:", str(e))
        return None

#Geleceğe yönelik tahminler için veri setini hazırlama ve feature eng
def prepare_time_series_data(ticker, start_date="7y"):
    stock = yf.Ticker(ticker)
    df = stock.history(period=start_date)
    if df.empty:
        raise ValueError(f"{ticker} için veri alınamadı. Lütfen geçerli bir hisse sembolü girin.")

    df = df.reset_index()
    df["Date"] = pd.to_datetime(df["Date"]).dt.date
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

    # Feature Engineering
    df["Percentage_Change"] = ((df['Close'] - df['Open']) / df['Open']) * 100
    df['Volume_Percentage_Change'] = df['Volume'].pct_change() * 100
    df.dropna(inplace=True)

    df['MA_7'] = df['Close'].rolling(window=7).mean()
    df['MA_30'] = df['Close'].rolling(window=30).mean()
    df['MA_60'] = df['Close'].rolling(window=60).mean()
    df['MA_90'] = df['Close'].rolling(window=90).mean()
    df['MA_180'] = df['Close'].rolling(window=180).mean()
    df["MA_365"] = df["Close"].rolling(window=365).mean()

    df['EWMA_7'] = df['Close'].ewm(span=7, adjust=False).mean()
    df['EWMA_30'] = df['Close'].ewm(span=30, adjust=False).mean()
    df['EWMA_60'] = df['Close'].ewm(span=60, adjust=False).mean()
    df['EWMA_90'] = df['Close'].ewm(span=90, adjust=False).mean()
    df['EWMA_180'] = df['Close'].ewm(span=180, adjust=False).mean()
    df['EWMA_365'] = df['Close'].ewm(span=365, adjust=False).mean()

    df = df.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"], axis=1)
    df.dropna(inplace=True)
    df = df.reset_index(drop=True)
    return df

#Pipeline ile Tahmin
def train_and_forecast(df, periods, investment_amount_usd, currency_rate):
    df.dropna(inplace=True)

    features = list(df.drop(columns=["Date", "Close"], axis=1).columns)
    target = 'Close'

    X = df[features]
    y = df[target]

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', Ridge())
    ])

    pipeline.fit(X, y)

    last_row = df.iloc[-1][features].values.reshape(1, -1)
    future_predictions = {}
    for period in periods:
        future_date = df['Date'].iloc[-1] + pd.Timedelta(days=period)
        predicted_price = pipeline.predict(last_row)[0]
        future_investment_usd = investment_amount_usd * (predicted_price / df['Close'].iloc[-1])
        future_investment_tl = future_investment_usd * currency_rate

        future_predictions[future_date.strftime('%Y-%m-%d')] = {
            "Tahmini Fiyat (USD)": predicted_price,
            "Yatırım Sonucu (USD)": future_investment_usd,
            "Yatırım Sonucu (TL)": future_investment_tl
        }
        last_row = np.roll(last_row, -1)
        last_row[0, -1] = predicted_price

    return future_predictions

# Main
def main():
    amount = float(input("Yatırım yapmak istediğiniz miktar: "))
    currency = input("Para biriminiz nedir? (TL/USD): ").strip().upper()

    if currency not in ["TL", "USD"]:
        print("Geçersiz para birimi girdiniz. Lütfen 'TL' veya 'USD' olarak giriniz.")
        return

    currency_rate = get_currency_rate()
    if not currency_rate:
        print("Döviz kuru alınamadı, program sonlandırılıyor.")
        return

    if currency == "TL":
        investment_amount_usd = amount / currency_rate
        other_currency_amount = investment_amount_usd
        other_currency = "USD"
    else:
        investment_amount_usd = amount
        other_currency_amount = amount * currency_rate
        other_currency = "TL"

    ticker = input("Hangi hisse senedine yatırım yapmak istiyorsunuz? (örn: AAPL): ").strip().upper()

    try:
        df = prepare_time_series_data(ticker)
        future_predictions = train_and_forecast(
            df,
            periods=[7, 30, 90, 180, 365],
            investment_amount_usd=investment_amount_usd,
            currency_rate=currency_rate
        )
    except ValueError as e:
        print(f"Hata: {e}")
        return

    periods = ["7d", "1mo", "3mo", "6mo", "1y"]
    stock_prices = {ticker: {period: get_stock_prices_history(ticker, period) for period in periods}}
    stock_results, amount_usd, amount_tl = calculate_stock_returns(amount, currency, stock_prices, currency_rate)

    # Çıktı Birleştirme
    print("\nYatırım Analiz ve Tahmin Raporu")

    print("\nGirdiğiniz Bilgiler:")
    print(f"  Yatırım Miktarı: {amount:.2f} {currency}")
    print(f"  {other_currency} Karşılığı: {other_currency_amount:.2f} {other_currency} (1 USD = {currency_rate:.2f} TL)")

    print("\nGeçmişten Günümüze Yatırım Analizi:")
    for ticker, data in stock_results.items():
        print(f"\nHisse: {ticker}")
        for period, result in data.items():
            date_range = get_period_date_range(period)
            print(f"  Dönem: {period} - {date_range}")
            if isinstance(result, str):
                print(f"    {result}")
            else:
                for key, value in result.items():
                    if isinstance(value, float):
                        print(f"    {key}: {value:.2f}")
                    else:
                        print(f"    {key}: {value}")

    print("\nGeleceğe Yönelik Tahminler:")
    for date, prediction in future_predictions.items():
        print(f"  Tarih: {date}")
        for key, value in prediction.items():
            if isinstance(value, float):
                print(f"    {key}: {value:.2f}")
            else:
                print(f"    {key}: {value}")


if __name__ == "__main__":
    main()


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names




Yatırım Analiz ve Tahmin Raporu

Girdiğiniz Bilgiler:
  Yatırım Miktarı: 1000.00 USD
  TL Karşılığı: 35681.53 TL (1 USD = 35.68 TL)

Geçmişten Günümüze Yatırım Analizi:

Hisse: NVDA
  Dönem: 7d - 17.01.2025 - 24.01.2025
    Başlangıç Fiyatı (USD): 131.76
    Bitiş Fiyatı (USD): 147.22
    Değişim Oranı (%): 11.73
    Sonuç (USD): 1117.33
    Sonuç (TL): 39868.21
  Dönem: 1mo - 25.12.2024 - 24.01.2025
    Başlangıç Fiyatı (USD): 140.22
    Bitiş Fiyatı (USD): 147.22
    Değişim Oranı (%): 4.99
    Sonuç (USD): 1049.92
    Sonuç (TL): 37462.81
  Dönem: 3mo - 26.10.2024 - 24.01.2025
    Başlangıç Fiyatı (USD): 140.40
    Bitiş Fiyatı (USD): 147.22
    Değişim Oranı (%): 4.86
    Sonuç (USD): 1048.57
    Sonuç (TL): 37414.69
  Dönem: 6mo - 28.07.2024 - 24.01.2025
    Başlangıç Fiyatı (USD): 114.23
    Bitiş Fiyatı (USD): 147.22
    Değişim Oranı (%): 28.88
    Sonuç (USD): 1288.78
    Sonuç (TL): 45985.53
  Dönem: 1y - 25.01.2024 - 24.01.2025
    Başlangıç Fiyatı (USD): 61.34
    Bitiş Fi

# Son Hale Görselleştirmenin de Eklenmiş Hali (Plotly ile, yatırılan paranın, yatırıldığı günden 1 yıl sonraki güne kadarki değişimi)